In [ ]:
# coding: UTF-8
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
from tkinter import scrolledtext 
from rdflib import Graph, RDFS, URIRef, Namespace, RDF, Literal

import networkx as nx
from PIL import Image, ImageTk
import matplotlib.pyplot as plt

import copy
import datetime
import re
import random
import string
import platform

import ScrollableFrame as scroll
import Neo4jConnection as neo4j
import FusekiManagement as fuseki
import DataManagement as data_mg
import Association as assoc
import setting

import importlib
importlib.reload(scroll)
importlib.reload(neo4j)
importlib.reload(fuseki)
importlib.reload(data_mg)
importlib.reload(assoc)
importlib.reload(setting)

In [ ]:
class Application(tk.Frame):
    """ メインアプリケーションクラス

    """
    # treeview Widgetの情報を格納する変数
    treeview = None

    def __init__(self, master=None):
        """  メインウィンドウの初期設定を行う

        Args:
            master:

        """
        super().__init__(master)
        self.master = master
        self.master.title("GPM Window")
        # メニューバー作成 
        self.set_menubar()
        self.frame = scroll.ScrollableFrame(self.master, canvas_width=1120, \
                                            canvas_height=585, frame_width=1260, \
                                            frame_height=760)
        self.frame.pack()
        # タブ設定
        self.set_tab()

        self.prefix_grpah_data = {}
        self.ttl_file = "ttl_file.ttl"
        self.selected_graph = None
        self.selected_graph_name = None
        self.gpm_uri = None
        self.select_edge_index = None
        self.graph_node_list = []
        self.graph_edge_list = []
        self.graph_label_list = {}
        self.graph_position = {}
        # ------------------------------
        # ログウィンドウ
        # ------------------------------
        # ログウインドウで選択したグラフ
        self.log_graph = tk.StringVar()
        # ログウインドウで選択したノード
        self.log_node = tk.StringVar()
        # ログウインドウで選択したuri
        self.log_uri = tk.StringVar()
        # ------------------------------
        # ウィンドウ2
        # ------------------------------
        # ウインドウ2で選択したグラフ
        self.win2_graph = tk.StringVar()
        # ウインドウ2で選択したノード
        self.win2_selected_item = tk.StringVar()
        # ウインドウ2で選択したuri
        self.win2_uri = tk.StringVar()

        # ------------------------------
        # オントロジーデータ
        # ------------------------------
        self.rootEntity = tk.StringVar()
        self.parentEntity = tk.StringVar()
        self.addEntity = tk.StringVar()
        self.savePathEntity = tk.StringVar()
        self.propName = tk.StringVar()
        self.propValue = tk.StringVar()
        self.relationName = tk.StringVar()
        self.filePath = tk.StringVar()
    
        self.savePathEntity.set("./saveRdfFile.ttl")
        self.iid = ""
        self.rootiid = ""
        self.ontologyTtlPath = "./fuseki_entity.ttl"
        self.entityGraph = "<http://localhost:3030/entity>"
        self.entityUri = "http://localhost/entity/"
        # tree iid table, format = {"top":[rootId], rootIid:[iid]}
        self.entityTree = {}
        self.entityTree["top"] = []
        # tree uri list, format = {"top:{rootName:uri}",rootName:{name:uri}}
        self.treeUriList = {}
        self.treeUriList['top'] = {}
        # tree name iid list, format = {"top:{rootName:iid}",rootName:{name:iid}}
        self.treeIdTable = {}
        self.treeIdTable['top'] = {}
        # Entity{propName:propValue}
        self.propInfo = {}
        # Entity{propName:propUri}
        self.propInfoUri = {}
        # [propName1, propName2, propName3...]
        self.propNameList = []

        self.relationInfoList = {}
        self.relationInfoListUri = {}
        self.relationUri = {}
        self.pd3 = Namespace('http://DigitalTriplet.net/2021/08/ontology#')
        self.fusekiInsert = []

        # 表示しているグラフの層を記録
        self.graph_level = None
        # Neo4jConnectionインスタンス生成
        self.sam = neo4j.Neo4jConnection(uri=setting.NEO4J_URI, \
                                        user=setting.NEO4J_USER,\
                                        pwd=setting.NEO4J_PW)
        self.data = data_mg.DataManagement()
        self.fuseki = fuseki.FusekiManagement(setting.FUSEKI_URL, \
                                            setting.FUSEKI_DB)

        # OSによってウィジェットの幅を調整する
        if platform.system() == "Windows":
            self.entryWidth = 14
            self.listBoxWidth = 26
        else:
            self.entryWidth = 9
            self.listBoxWidth = 21

        self.create_widgets()

    def set_menubar(self):
        """  メニューバー作成

        """
        menubar = tk.Menu(self.master) 
        root.config(menu=menubar) 
        # Functionメニュー
        menu_function = tk.Menu(self.master, tearoff=0) 
        menubar.add_cascade(label="Execute", menu=menu_function)
        con_menu = tk.Menu(menu_function, tearoff=0)
        menu_function.add_cascade(label="コンテナ関連付け",menu=con_menu)
        con_menu.add_command(label="Open Window", command=self.open_window_2)
        con_menu.add_separator() 
        con_menu.add_command(label="EP全体をメンバー関連付け追加", \
                                    command=self.insert_ep_member_event)
        con_menu.add_command(label="コンテナのみをメンバー関連付け追加", \
                                    command=self.insert_con_member_event)
        con_menu.add_command(label="削除", command=self.delete_member_event)

        use_menu = tk.Menu(menu_function, tearoff=0)
        menu_function.add_cascade(label="Use関連付け",menu=use_menu)
        use_menu.add_command(label="Open Log Window", command=self.open_log_window)
        use_menu.add_separator() 
        use_menu.add_command(label="追加", command=self.insert_use_event)
        use_menu.add_command(label="削除", command=self.delete_use_event)

        # オントロジー操作メニュー
        ontology_menu = tk.Menu(menu_function, tearoff=0)
        menu_function.add_cascade(label="Ontology関連付け",menu=ontology_menu)
        ontology_menu.add_command(label="Input 追加", command=self.insertInput)
        ontology_menu.add_command(label="Input 削除", command=self.delInput)
        ontology_menu.add_separator() 
        ontology_menu.add_command(label="Output 追加", command=self.insertOutput)
        ontology_menu.add_command(label="Output 削除", command=self.delOutput)

        # Searchメニュー
        menu_search = tk.Menu(self.master, tearoff=0) 
        menubar.add_cascade(label="Search", menu=menu_search)
        menu_search.add_command(label='知識文書検索', command= \
                                self.insert_document_list)
        menu_search.add_command(label='技術者検索', command= \
                                self.insert_engineer_list)
        menu_search.add_command(label='過去事例検索', command= \
                                self.insert_lld_list)
        menu_search.add_command(label='SAM因子検索', command= \
                                self.insert_samfactor_list)

        # Exportメニュー
        menu_export = tk.Menu(self.master, tearoff=0) 
        menubar.add_cascade(label="Export", menu=menu_export)
        menu_export.add_command(label='Graph Export', command=lambda: \
                                self.fuseki.ttlfile_export(self))
        menu_export.add_command(label='Fuseki all Export', command=lambda: \
                                self.fuseki.ttlfile_export_all(self)) 

    def set_tab(self):
        """  タブ作成

        """
        style = ttk.Style()
        style.configure('One.TNotebook.Tab',font=('Arial', 8), padding=0)

        book_graph = ttk.Notebook(self.frame.scrollable_frame, \
                                style='One.TNotebook', width=650)
        book_graph.place(x=0, y=0)
        self.frame_tab_graph = ttk.Frame(book_graph, width=650, \
                                        height=560, relief="groove")
        book_graph.add(self.frame_tab_graph, text=" Graph ")

        self.book_action = ttk.Notebook(self.frame.scrollable_frame, \
                                        style='Ono.TNotebook', width=470, height=560)
        self.book_action.place(x=650, y=-1)
        self.frame_tab_action = ttk.Frame(self.book_action, width=470, \
                                        height=580, relief="groove")
        self.book_action.add(self.frame_tab_action, text='Action')
        self.frame_tab_edge = ttk.Frame(self.book_action, width=470, \
                                        height=580, relief="groove")
        self.book_action.add(self.frame_tab_edge , text='Edge')
        self.frame_tab_search = ttk.Frame(self.book_action, width=470, \
                                        height=580, relief="groove")
        self.book_action.add(self.frame_tab_search , text='Search')
        self.frame_tab_ontology = ttk.Frame(self.book_action, width=470, \
                                            height=580, relief="groove")
        self.book_action.add(self.frame_tab_ontology , text='Ontology')

    def create_widgets(self):
        """ ウィジェットを作成

        """
        # ------------------------------------------------------------------
        # LabelFrame
        # ------------------------------------------------------------------
        self.create_frame_widget()
        # ------------------------------------------------------------------
        # listbox widget (Graph Info)
        # ------------------------------------------------------------------
        self.scrollbar_ep_info_x = ttk.Scrollbar(self.frame_ep_info_list, \
                                                orient=tk.HORIZONTAL)
        self.scrollbar_ep_info_x.pack(side="bottom", fill="x")
        self.scrollbar_ep_info_y = ttk.Scrollbar(self.frame_ep_info_list, \
                                                orient=tk.VERTICAL)
        self.scrollbar_ep_info_y.pack(side="right", fill="y")

        self.listbox_ep_value = tk.StringVar()
        self.listbox_ep_info = tk.Listbox(self.frame_ep_info_list, \
                                        height=12, width=23, \
                                        selectmode=tk.SINGLE, \
                                        listvariable=self.listbox_ep_value, \
                                        exportselection=False, \
                                        xscrollcommand=self.scrollbar_ep_info_x.set, \
                                        yscrollcommand=self.scrollbar_ep_info_y.set)
        self.listbox_ep_info.pack(fill="both")
        self.listbox_ep_info.bind("<<ListboxSelect>>", self.select_graph)

        self.scrollbar_ep_info_x.config(command=self.listbox_ep_info.xview)
        self.scrollbar_ep_info_y.config(command=self.listbox_ep_info.yview)
        # ------------------------------------------------------------------
        # treeview widget
        # ------------------------------------------------------------------
        self.treeview = ttk.Treeview(self.frame_hierarchy_list, height=11)
        # 階層列設定
        self.treeview.column("#0", width=160, minwidth=500)
        # 階層列(ツリーカラム)の見出し設定。階層列とする。
        self.treeview.heading("#0")
        self.treeview.bind("<<TreeviewSelect>>", self.tree_select)
        self.treeview.grid(row=1, column=1, sticky="nsew")

        tree_scrollbar_y = ttk.Scrollbar(self.frame_hierarchy_list, \
                                        orient="vertical", \
                                        command=self.treeview.yview)
        tree_scrollbar_y.grid(row=1, column=2, sticky="nsew")
        self.treeview.configure(yscrollcommand=tree_scrollbar_y.set)

        tree_scrollbar_x = ttk.Scrollbar(self.frame_hierarchy_list, \
                                        orient="horizontal", \
                                        command=self.treeview.xview)
        tree_scrollbar_x.grid(row=2, column=1, sticky="nsew")
        self.treeview.configure(xscrollcommand=tree_scrollbar_x.set)
        # ------------------------------------------------------------------
        # ノード情報ウィジェットを作成
        # ------------------------------------------------------------------
        self.create_node_info_widget()
        # ------------------------------------------------------------------
        # listbox widget (GPM uri Info)
        # ------------------------------------------------------------------
        self.scrollbar_gpm_uri_list_x = ttk.Scrollbar(self.frame_new_gpm_list, \
                                                    orient=tk.HORIZONTAL)
        self.scrollbar_gpm_uri_list_x.pack(side="bottom", fill="x")
        self.scrollbar_gpm_uri_list_y = ttk.Scrollbar(self.frame_new_gpm_list, \
                                                    orient=tk.VERTICAL)
        self.scrollbar_gpm_uri_list_y.pack(side="right", fill="y")

        self.listbox_gpm_uri_list = tk.Listbox(self.frame_new_gpm_list, \
                                            height=12, width=23, \
                                            selectmode=tk.SINGLE, \
                                            exportselection=False, \
                                            xscrollcommand=self.scrollbar_gpm_uri_list_x.set, \
                                            yscrollcommand=self.scrollbar_gpm_uri_list_y.set)
        self.listbox_gpm_uri_list.pack(fill="both")

        self.scrollbar_gpm_uri_list_x.config(command=self.listbox_gpm_uri_list.xview)
        self.scrollbar_gpm_uri_list_y.config(command=self.listbox_gpm_uri_list.yview)
        # ------------------------------------------------------------------
        # listbox widget (object list)
        # ------------------------------------------------------------------
        self.scrollbar_object_x = ttk.Scrollbar(self.frame_object_list, \
                                            orient=tk.HORIZONTAL)
        self.scrollbar_object_x.pack(side="bottom", fill="x")
        self.scrollbar_object_y = ttk.Scrollbar(self.frame_object_list, \
                                            orient=tk.VERTICAL)
        self.scrollbar_object_y.pack(side="right", fill="y")

        self.listbox_object = tk.Listbox(self.frame_object_list, height=7, \
                                        width=self.listBoxWidth, \
                                        selectmode=tk.SINGLE, exportselection=False, \
                                        xscrollcommand=self.scrollbar_object_x.set, \
                                        yscrollcommand=self.scrollbar_object_y.set)
        self.listbox_object.pack(fill="both")
        self.listbox_object.bind("<<ListboxSelect>>", self.select_object)

        self.scrollbar_object_x.config(command=self.listbox_object.xview)
        self.scrollbar_object_x.config(command=self.listbox_object.yview)
        # ------------------------------------------------------------------
        # listbox widget (Document)
        # ------------------------------------------------------------------
        self.scrollbar_document_x = ttk.Scrollbar(self.frame_document_list, \
                                                orient=tk.HORIZONTAL)
        self.scrollbar_document_x.pack(side="bottom", fill="x")
        self.scrollbar_document_y = ttk.Scrollbar(self.frame_document_list, \
                                                orient=tk.VERTICAL)
        self.scrollbar_document_y.pack(side="right", fill="y")

        self.listbox_document = tk.Listbox(self.frame_document_list, \
                                        height=12, width=23, \
                                        selectmode=tk.SINGLE, listvariable='', \
                                        exportselection=False, \
                                        xscrollcommand=self.scrollbar_document_x.set, \
                                        yscrollcommand=self.scrollbar_document_y.set)
        self.listbox_document.pack(fill="both")

        self.scrollbar_document_x.config(command=self.listbox_document.xview)
        self.scrollbar_document_y.config(command=self.listbox_document.yview)
        # ------------------------------------------------------------------
        # listbox widget (Engineer)
        # ------------------------------------------------------------------
        self.scrollbar_engineer_x = ttk.Scrollbar(self.frame_engineer_list, \
                                                orient=tk.HORIZONTAL)
        self.scrollbar_engineer_x.pack(side="bottom", fill="x")
        self.scrollbar_engineer_y = ttk.Scrollbar(self.frame_engineer_list, \
                                                orient=tk.VERTICAL)
        self.scrollbar_engineer_y.pack(side="right", fill="y")

        self.listbox_engineer = tk.Listbox(self.frame_engineer_list, \
                                        height=12, width=23, \
                                        selectmode=tk.SINGLE, listvariable='', \
                                        exportselection=False, \
                                        xscrollcommand=self.scrollbar_engineer_x.set, \
                                        yscrollcommand=self.scrollbar_engineer_y.set)
        self.listbox_engineer.pack(fill="both")

        self.scrollbar_engineer_x.config(command=self.listbox_engineer.xview)
        self.scrollbar_engineer_y.config(command=self.listbox_engineer.yview)
        # ------------------------------------------------------------------
        # listbox widget (LLD)
        # ------------------------------------------------------------------
        self.scrollbar_lld_x = ttk.Scrollbar(self.frame_lld_list, \
                                            orient=tk.HORIZONTAL)
        self.scrollbar_lld_x.pack(side="bottom", fill="x")
        self.scrollbar_lld_y = ttk.Scrollbar(self.frame_lld_list, \
                                            orient=tk.VERTICAL)
        self.scrollbar_lld_y.pack(side="right", fill="y")

        self.listbox_lld_value = tk.StringVar()
        self.listbox_lld = tk.Listbox(self.frame_lld_list, \
                                    height=12, width=23, \
                                    selectmode=tk.SINGLE, listvariable='', \
                                    exportselection=False, \
                                    xscrollcommand=self.scrollbar_lld_x.set, \
                                    yscrollcommand=self.scrollbar_lld_y.set)
        self.listbox_lld.pack(fill="both")

        self.scrollbar_lld_x.config(command=self.listbox_lld.xview)
        self.scrollbar_lld_y.config(command=self.listbox_lld.yview)
        # ------------------------------------------------------------------
        # listbox widget (SAM factor)
        # ------------------------------------------------------------------
        self.scrollbar_samfactor_x = ttk.Scrollbar(self.frame_samfactor_list, \
                                            orient=tk.HORIZONTAL)
        self.scrollbar_samfactor_x.pack(side="bottom", fill="x")
        self.scrollbar_samfactor_y = ttk.Scrollbar(self.frame_samfactor_list, \
                                            orient=tk.VERTICAL)
        self.scrollbar_samfactor_y.pack(side="right", fill="y")

        self.listbox_samfactor = tk.Listbox(self.frame_samfactor_list, \
                                    height=12, width=23, \
                                    selectmode=tk.SINGLE, listvariable='', \
                                    exportselection=False, \
                                    xscrollcommand=self.scrollbar_samfactor_x.set, \
                                    yscrollcommand=self.scrollbar_samfactor_y.set)
        self.listbox_samfactor.pack(fill="both")

        self.scrollbar_samfactor_x.config(command=self.listbox_samfactor.xview)
        self.scrollbar_samfactor_y.config(command=self.listbox_samfactor.yview)
        # ------------------------------------------------------------------
        # listbox widget (edge list)
        # ------------------------------------------------------------------
        self.scrollbar_edge_x = ttk.Scrollbar(self.frame_edge_list, \
                                            orient=tk.HORIZONTAL)
        self.scrollbar_edge_x.pack(side="bottom", fill="x")
        self.scrollbar_edge_y = ttk.Scrollbar(self.frame_edge_list, \
                                            orient=tk.VERTICAL)
        self.scrollbar_edge_y.pack(side="right", fill="y")

        self.listbox_edge = tk.Listbox(self.frame_edge_list, \
                                    height=12, width=40, \
                                    selectmode=tk.SINGLE, listvariable='', \
                                    exportselection=False, \
                                    xscrollcommand=self.scrollbar_edge_x.set, \
                                    yscrollcommand=self.scrollbar_edge_y.set)
        self.listbox_edge.pack(fill="both")
        self.listbox_edge.bind("<<ListboxSelect>>", self.select_edge)

        self.scrollbar_edge_x.config(command=self.listbox_edge.xview)
        self.scrollbar_edge_y.config(command=self.listbox_edge.yview)
        # ------------------------------------------------------------------
        # Button widget
        # ------------------------------------------------------------------
        # 表示EPを設定
        self.button_gpm_set = ttk.Button(self.frame_button, text=" 表示 ", \
                                        width=8, \
                                        command=self.set_gpm)
        self.button_gpm_set.grid(row=1, column=0, padx=5)
        # ------------------------------------------------------------------
        # label
        # ------------------------------------------------------------------
        self.label_log_graph = ttk.Label(self.frame_button, textvariable=self.log_graph)
        self.label_log_uri = ttk.Label(self.frame_button, textvariable=self.log_uri)
        self.label_log_node = ttk.Label(self.frame_button, textvariable=self.log_node)
        # ------------------------------------------------------------------
        # canvas
        # ------------------------------------------------------------------
        self.graph_canvas = tk.Canvas(self.frame_tab_graph)
        self.graph_canvas.place(x=5, y=5, width=455, height=550)
        # canvas スクロールバー
        self.canvas_bar_y = ttk.Scrollbar(self.graph_canvas, orient=tk.VERTICAL)
        self.canvas_bar_x = ttk.Scrollbar(self.graph_canvas, orient=tk.HORIZONTAL)
        self.canvas_bar_y.pack(side=tk.RIGHT, fill=tk.Y)
        self.canvas_bar_x.pack(side=tk.BOTTOM, fill=tk.X)
        self.canvas_bar_y.config(command=self.graph_canvas.yview)
        self.canvas_bar_x.config(command=self.graph_canvas.xview)
        self.graph_canvas.config(yscrollcommand=self.canvas_bar_y.set, \
                                xscrollcommand=self.canvas_bar_x.set)
        # Canvasのスクロール範囲を設定
        self.graph_canvas.config(scrollregion=(5, 5, 1000, 1000))

        # graphを取得し、リストボックスに設定
        (self.graph_url_list, self.graph_list) = \
                                self.fuseki.get_graph(setting.FUSEKI_URL + "/")
        self.listbox_ep_value.set(self.graph_list)

        # Action output
        self.node_info_input = ttk.Label(self.frame_ontology, \
                                            text="Action\nInput: ")
        self.node_info_input.grid(row=0, column=0)
        self.actionInput_info = scrolledtext.ScrolledText(self.frame_ontology, \
                                                    width=18, height=1, wrap=tk.WORD)
        self.actionInput_info.grid(row=0, column=1)
        self.actionInput_info.configure(state="disabled")

        # Action output
        self.node_info_output = ttk.Label(self.frame_ontology, \
                                    text="Action \nOutput:")
        self.node_info_output.grid(row=0, column=2)
        self.actionOutput_info = scrolledtext.ScrolledText(self.frame_ontology, \
                                                width=25, height=1, \
                                                wrap=tk.WORD)
        self.actionOutput_info.grid(row=0, column=3)
        self.actionOutput_info.configure(state="disabled")

        # Ontology使用widgetsを作成
        self.create_ontology_widgets()

    def create_frame_widget(self):
        """ LabelFrameを作成

        """
        # graph info
        self.frame_ep_info_list = ttk.LabelFrame(self.frame_tab_graph, \
                                                text=" Graph Info", \
                                                height=120, \
                                                style="style.TFrame")
        self.frame_ep_info_list.place(x=465, y=5)
        # Top Level URI List
        self.frame_new_gpm_list = ttk.LabelFrame(self.frame_tab_graph, \
                                                text="Top Level EP List", \
                                                height=200, \
                                                style="style.TFrame")
        self.frame_new_gpm_list.place(x=465, y=240)
        # Button
        self.frame_button = ttk.LabelFrame(self.frame_tab_graph, \
                                        text="Button", \
                                        width=250, height=70, \
                                        style="style.TFrame")
        self.frame_button.place(x=465, y=480)
        # node list
        self.frame_hierarchy_list = ttk.LabelFrame(self.frame_tab_action, \
                                                text="Action List", \
                                                style="style.TFrame")
        self.frame_hierarchy_list.grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        # node info
        self.frame_node = ttk.LabelFrame(self.frame_tab_action, \
                                        text="Information", \
                                        style="style.TFrame")
        self.frame_node.grid(row=0, column=1, rowspan=2, pady=5, sticky=tk.N)
        # object表示frame
        self.frame_object_list = ttk.LabelFrame(self.frame_tab_action, \
                                            text="Object List", \
                                            width=160, \
                                            height=150, \
                                            style="style.TFrame")
        self.frame_object_list.grid(row=1, column=0, padx=5, sticky=tk.W)
        # オントロジーデータ
        self.frame_ontology = ttk.LabelFrame(self.frame_tab_action, \
                                            text="Action Input/Output", \
                                            style="style.TFrame")
        self.frame_ontology.grid(row=2, column=0, columnspan=2, pady=10)

        # 知識文書リスト
        self.frame_document_list = ttk.LabelFrame(self.frame_tab_search, \
                                                text="Document List", \
                                                width=150, \
                                                height=140, \
                                                style="style.TFrame")
        self.frame_document_list.grid(row=0, column=0,  padx=5, pady=10)
        # 技術者リスト
        self.frame_engineer_list = ttk.LabelFrame(self.frame_tab_search, \
                                                text="Engineer List", \
                                                width=150, \
                                                height=140, \
                                                style="style.TFrame")
        self.frame_engineer_list.grid(row=0, column=1)
        # 過去事例リスト
        self.frame_lld_list = ttk.LabelFrame(self.frame_tab_search, \
                                            text="LLD List", \
                                            width=150, \
                                            height=140, \
                                            style="style.TFrame")
        self.frame_lld_list.grid(row=1, column=0)
        # SAM因子リスト
        self.frame_samfactor_list = ttk.LabelFrame(self.frame_tab_search, \
                                            text="SAM Factor List", \
                                            width=150, height=140, \
                                            style="style.TFrame")
        self.frame_samfactor_list.grid(row=1, column=1)

        # エッジリスト
        self.frame_edge_list = ttk.LabelFrame(self.frame_tab_edge, \
                                            text="Edge List", \
                                            width=150, height=140, \
                                            style="style.TFrame")
        self.frame_edge_list.grid(row=0, column=0,  padx=5, pady=10)

    def create_node_info_widget(self):
        """ ノード情報ウィジェットを作成

        """
        # ------------------------------------------------------------------
        # ノード情報
        # ------------------------------------------------------------------
        # id
        self.node_info_id = ttk.Label(self.frame_node, \
                                    text="Id:                 ")
        self.node_info_id.grid(row=0, column=0)
        self.id_info = tk.Text(self.frame_node, width=25, height=1)
        self.id_info.grid(row=0, column=1)
        self.id_info.configure(state="disabled")
        # action
        self.node_info_actionType = ttk.Label(self.frame_node, \
                                            text="ActionType:")
        self.node_info_actionType.grid(row=1, column=0)
        self.action_info = tk.Text(self.frame_node, width=25, \
                                    height=1, wrap=tk.WORD)
        self.action_info.grid(row=1, column=1)
        self.action_info.configure(state="disabled")
        # layer
        self.node_info_layer = ttk.Label(self.frame_node, \
                                        text="Layer:          ")
        self.node_info_layer.grid(row=2, column=0)
        self.layer_info = tk.Text(self.frame_node, width=25, height=1, \
                                    wrap=tk.WORD)
        self.layer_info.grid(row=2, column=1)
        self.layer_info.configure(state="disabled")
        # value
        self.node_info_value = ttk.Label(self.frame_node, \
                                        text="Value:           ")
        self.node_info_value.grid(row=3, column=0)
        self.value_info = scrolledtext.ScrolledText(self.frame_node, width=23, \
                                                    height=1, wrap=tk.WORD)
        self.value_info.grid(row=3, column=1)
        self.value_info.configure(state="disabled")
        # use
        self.node_info_use = ttk.Label(self.frame_node, \
                                        text="Use:              ")
        self.node_info_use.grid(row=4, column=0)
        self.use_info = scrolledtext.ScrolledText(self.frame_node, width=23, \
                                                height=1, wrap=tk.WORD)
        self.use_info.grid(row=4, column=1)
        self.use_info.configure(state="disabled")
        # useBy
        self.node_info_useby = ttk.Label(self.frame_node, \
                                        text="UseBy:          ")
        self.node_info_useby.grid(row=5, column=0)
        self.useby_info = scrolledtext.ScrolledText(self.frame_node, width=23, \
                                                    height=1, wrap=tk.WORD)
        self.useby_info.grid(row=5, column=1)
        self.useby_info.configure(state="disabled")
        # ------------------------------------------------------------------
        # 補足情報
        # ------------------------------------------------------------------
        # Intention
        self.node_info_intention = ttk.Label(self.frame_node, \
                                            text="Intention:    ")
        self.node_info_intention.grid(row=6, column=0)
        self.intention_info = scrolledtext.ScrolledText(self.frame_node, \
                                                        width=23, height=1, \
                                                        wrap=tk.WORD)
        self.intention_info.grid(row=6, column=1)
        self.intention_info.configure(state="disabled")

        # Rationale
        self.node_info_rationale = ttk.Label(self.frame_node, \
                                            text="Rationale:    ")
        self.node_info_rationale.grid(row=7, column=0)
        self.rationale_info = scrolledtext.ScrolledText(self.frame_node, \
                                                        width=23, height=1, \
                                                        wrap=tk.WORD)
        self.rationale_info.grid(row=7, column=1)
        self.rationale_info.configure(state="disabled")

        # Annotation
        self.node_info_annotation = ttk.Label(self.frame_node, \
                                            text="Annotation: ")
        self.node_info_annotation.grid(row=8, column=0)
        self.annotation_info = scrolledtext.ScrolledText(self.frame_node, \
                                                        width=23, height=1, \
                                                        wrap=tk.WORD)
        self.annotation_info.grid(row=8, column=1)
        self.annotation_info.configure(state="disabled")

        # Tool
        self.node_info_tool = ttk.Label(self.frame_node, \
                                    text="Tool:   \t      ")
        self.node_info_tool.grid(row=9, column=0)
        self.tool_info = scrolledtext.ScrolledText(self.frame_node, width=23, \
                                                height=1, wrap=tk.WORD)
        self.tool_info.grid(row=9, column=1)
        self.tool_info.configure(state="disabled")

    def create_ontology_widgets(self):
        """ オントロジーウィジェットを作成

        """
        # オントロジー表示ツリーリスト
        entityFrame = self.createEntityTreeView()
        entityFrame.grid(column=0, row=0, padx=5, pady=10, sticky=tk.W)
        # オントロジー操作
        entityEditFrame = self.createInputPanel()
        entityEditFrame.grid(column=1, row=0, sticky=tk.W)
        # パラメータ表示と操作
        propertyEditFrame = self.createPropEditPanel()
        propertyEditFrame.grid(column=0, row=1, padx=5, sticky=tk.W)
        # リレーション表示と操作
        relationshipEditFrame = self.createRelationshipPanel()
        relationshipEditFrame.grid(column=1, row=1, sticky=tk.W)

    def createEntityTreeView(self):
        """ オントロジーツリーウィジェットを作成

        """
        treeFrame = ttk.LabelFrame(self.frame_tab_ontology, \
                                    text="Ontology Tree", \
                                    style="style.TFrame")
        self.tree = ttk.Treeview(treeFrame, height=6)
        self.tree.column("#0", width=200, minwidth=500)
        self.tree.bind("<<TreeviewSelect>>", self.targetItem)
        self.tree.heading("#0")
        self.tree.grid(row=1, column=1, sticky="nsew")

        tree_scrollbar_x = ttk.Scrollbar(treeFrame, orient="horizontal", \
                                        command=self.tree.xview)
        tree_scrollbar_x.grid(row=2, column=1, sticky="nsew")
        self.tree.configure(xscrollcommand=tree_scrollbar_x.set)

        # fusekiからentity dataを取得
        self.fuseki.get_graph_ttlfile(self.entityGraph, self.ontologyTtlPath)
        # entity treeviewを作成
        self.readRDFfile(self.ontologyTtlPath)

        return treeFrame

    def createInputPanel(self):
        """ オントロジーツリー操作ウィジェットを作成

        """
        inputFrame = ttk.LabelFrame(self.frame_tab_ontology, \
                                    text="EditParam", \
                                    style="style.TFrame")

        rootLabel = ttk.Label(inputFrame, text="RootEntity")
        rootLabel.grid(column=0, row=0)
        addrootEntry = ttk.Entry(inputFrame, width=self.entryWidth, \
                                textvariable=self.rootEntity)
        addrootEntry.grid(column=1, row=0)
        addrootButton = ttk.Button(inputFrame, width=8, text="AddRoot", \
                                    command=self.insertRootEntity)
        addrootButton.grid(column=2, row=0)

        addPathLabel = ttk.Label(inputFrame, text="TargetEntity")
        addPathLabel.grid(column=0, row = 1)
        addpathEntry = ttk.Entry(inputFrame, width=self.entryWidth, \
                                textvariable=self.parentEntity)
        addpathEntry.grid(column=1, row=1)
        addPathButton = ttk.Button(inputFrame, width=8, text="Delete", \
                                    command=self.deleteEntity)
        addPathButton.grid(column=2 , row=1)

        editButton = ttk.Button(inputFrame, width=8, text="Edit", \
                                command=self.editEntity)
        editButton.grid(column=2 , row=2)

        addNodeLabel = ttk.Label(inputFrame, text="AddEntity")
        addNodeLabel.grid(column=0, row=3)
        addNodeEntry = ttk.Entry(inputFrame, width=self.entryWidth, \
                                textvariable=self.addEntity)
        addNodeEntry.grid(column=1, row=3)
        addNodeButton = ttk.Button(inputFrame, width=8, text="Add", \
                                    command=self.insertEntity)
        addNodeButton.grid(column=2, row=3)

        savePathLabel = ttk.Label(inputFrame, text="SavePath")
        savePathLabel.grid(column=0, row=4)
        savePathEntry = ttk.Entry(inputFrame, width=self.entryWidth, \
                                textvariable=self.savePathEntity)
        savePathEntry.grid(column=1, row=4)
        saveButton = ttk.Button(inputFrame, width=8, text="SaveFile", \
                                command=self.save)
        saveButton.grid(column=2, row=4)

        PathLabel = ttk.Label(inputFrame,text="OpenPath")
        PathLabel.grid(column=0,row=5)
        pathEntry = ttk.Entry(inputFrame, width=self.entryWidth, \
                            textvariable=self.filePath)
        pathEntry.grid(column=1,row=5)
        #pathButton = ttk.Button(inputFrame, width=8, text="OpenFile", \
        #                       command=self.openFileDialog)
        pathButton = ttk.Button(inputFrame, width=8, text="OpenFile")
        pathButton.grid(column=2,row=5)
        pathButton['state'] = 'disabled'

        return inputFrame

    def createPropEditPanel(self):
        """ プロパティ表示と操作ウィジェットを作成

        """
        propFrame = ttk.LabelFrame(self.frame_tab_ontology, \
                                height=200, text="Property EditParam",\
                                style="style.TFrame")

        self.listboxPropValue = tk.StringVar()
        self.propList = tk.Listbox(propFrame, height=12, \
                                selectmode=tk.SINGLE, \
                                listvariable=self.listboxPropValue, \
                                exportselection=False)
        self.propList.grid(column=0, row=0, columnspan=2, sticky=tk.W + tk.E)
        self.propList.bind("<<ListboxSelect>>", self.selectProp)

        addPropLabel = ttk.Label(propFrame, text="Property\nName")
        addPropLabel.grid(column=0, row=1)
        self.addPropNameEntry = ttk.Entry(propFrame, width=self.entryWidth, \
                                        textvariable=self.propName)
        self.addPropNameEntry.grid(column=1, row=1)
        addPropNameButton = ttk.Button(propFrame, width=8, text="Add", \
                                    command=self.addProp)
        addPropNameButton.grid(column=2, row=1)
        editPropNameButton = ttk.Button(propFrame, width=8, text="Edit", \
                                        command=self.editProp)
        editPropNameButton.grid(column=2, row=2)
        delPropNameButton = ttk.Button(propFrame, width=8, text="Delete", \
                                    command=self.deleteProp)
        delPropNameButton.grid(column=2, row=3)

        addPropValueLabel = ttk.Label(propFrame, text="Property\nValue")
        addPropValueLabel.grid(column=0, row=4)
        self.addPropValueEntry = ttk.Entry(propFrame, width=self.entryWidth, \
                                        textvariable=self.propValue)
        self.addPropValueEntry.grid(column=1, row=4)      
        addPropValueButton = ttk.Button(propFrame, width=8, text="Edit", \
                                        command=self.editPropvalue)
        addPropValueButton.grid(column=2, row=4)

        return propFrame      

    def createRelationshipPanel(self):
        """ リレーション表示と操作ウィジェットを作成

        """
        relationshipFrame = ttk.LabelFrame(self.frame_tab_ontology, \
                                            height=200, \
                                            text="Relationship EditParam",\
                                            style="style.TFrame")

        # relation listを作成
        relationListFrame = tk.LabelFrame(relationshipFrame)
        relationListFrame.grid(column=0, row=0, columnspan=2, sticky=tk.W + tk.E)

        self.listboxRelation = tk.StringVar()
        self.relationList = tk.Listbox(relationListFrame, 
                                        height=7, width=23, \
                                        selectmode=tk.SINGLE, \
                                        listvariable=self.listboxRelation, \
                                        exportselection=False)
        self.relationList.grid(column=0, row=0)
        self.relationList.bind("<<ListboxSelect>>", self.selectRelation)

        scrollbar_y = ttk.Scrollbar(relationListFrame, \
                                    orient=tk.VERTICAL, \
                                    command=self.relationList.yview)
        self.relationList['yscrollcommand'] = scrollbar_y.set
        scrollbar_y.grid(column=2, row=0, sticky=tk.N + tk.S + tk.E)

        scrollbar_x = ttk.Scrollbar(relationListFrame, \
                                    orient=tk.HORIZONTAL, \
                                    command=self.relationList.xview)
        self.relationList['xscrollcommand'] = scrollbar_x.set
        scrollbar_x.grid(column=0, row=0, columnspan=2, sticky=tk.S + tk.W + tk.E)

        #fusekiにある関係を取得
        self.initRelation()

        addrelationLabel = ttk.Label(relationshipFrame, text="Relationship")
        addrelationLabel.grid(column=0, row=1)
        self.addRelationEntry = ttk.Entry(relationshipFrame, \
                                        width=self.entryWidth, \
                                        textvariable=self.relationName)
        self.addRelationEntry.grid(column=1, row=1)
        self.addPropNameEntry.grid(column=1, row=1)
        addRelationButton = ttk.Button(relationshipFrame, width=8, \
                                        text="Add", command=self.addRelation)
        addRelationButton.grid(column=2, row=1)
        editRelationButton = ttk.Button(relationshipFrame, width=8, \
                                        text="Edit", command=self.editRelation)
        editRelationButton.grid(column=2, row=2)
        delRelationButton = ttk.Button(relationshipFrame, width=8, \
                                        text="Delete", command=self.delRelation)
        delRelationButton.grid(column=2, row=3)

        relationLabel = ttk.Label(relationshipFrame, text="Parameter")
        relationLabel.grid(column=0, row=4)

        # relation param listを作成
        relationParamsFrame = tk.LabelFrame(relationshipFrame)
        relationParamsFrame.grid(column=0, row=5, columnspan=2, sticky=tk.W + tk.E)

        self.listboxRelationParams = tk.StringVar()
        self.relationParamsList = tk.Listbox(relationParamsFrame, \
                                            height=6, \
                                            width=23, \
                                            selectmode=tk.SINGLE, \
                                            listvariable=self.listboxRelationParams, \
                                            exportselection=False)
        self.relationParamsList.grid(column=0, row=0)

        paramsScrollbar_y = ttk.Scrollbar(relationParamsFrame, \
                                        orient=tk.VERTICAL, \
                                        command=self.relationParamsList.yview)
        self.relationParamsList['yscrollcommand'] = paramsScrollbar_y.set
        paramsScrollbar_y.grid(column=2, row=0, sticky=tk.W + tk.N + tk.S)

        paramsScrollbar_x = ttk.Scrollbar(relationParamsFrame, \
                                        orient=tk.HORIZONTAL, \
                                        command=self.relationParamsList.xview)
        self.relationParamsList['xscrollcommand'] = paramsScrollbar_x.set
        paramsScrollbar_x.grid(column=0, row=0, columnspan=2, sticky=tk.S + tk.W + tk.E)

        frame = ttk.LabelFrame(relationshipFrame)
        frame.grid(column=2, row=5, sticky=tk.N)
        addRelationParamsButton = ttk.Button(frame, width=8, text="Add", \
                                            command=self.addRelationParams)
        addRelationParamsButton.pack()
        delRelationParamsButton = ttk.Button(frame, width=8, text="Delete", \
                                            command=self.delRelationParams)
        delRelationParamsButton.pack()

        return relationshipFrame

    def get_hierarchy_node_list(self, GPM_uri):
        """ ツリーリストの階層ノード情報設定

        Args:
            GPM_uri: 選択したGPMのuri

        Returns:

        """
        # グラフにあるコンテナフローを取得
        container_count = self.data.get_graph_containerflow_count(
            self.selected_graph_name, GPM_uri)

        (self.node_id_list) = self.data.get_top_node_list_in_uri(
            self.selected_graph_name, GPM_uri)

        (self.object_list, self.object_id_list) = self.data.get_top_level_object_list(
            self.selected_graph_name, self.gpm_uri)

        (self.edge_list, self.edge_id_list) = self.data.get_top_level_edge_list(
            self.selected_graph_name, self.gpm_uri)

        self.node_list = {}
        self.node_list["top"] = []
        self.treeview_item_list = {}
        self.treeview_item_list["top"] = []

        for node_id in self.node_id_list["top"]:
            node_value = self.data.get_node_value(self.selected_graph_name, GPM_uri, node_id)
            self.node_list["top"].append(node_value)

        for index, node in enumerate(self.node_list["top"]):
            parent = self.treeview.insert("", tk.END, text=node, open=False)
            self.treeview_item_list["top"].append(parent)
            self.get_member_node_list(
                GPM_uri, self.node_id_list["top"][index], container_count, parent)

    def get_member_node_list(self, uri, top_node, count, parent_node):
        """ ツリーリストのmember情報を取得

        Args:
            uri: 選択したGPMのuri
            top_node: member情報を取得したいトップノードのid
            count: GPMにあるコンテナ数
            parent_node: ツリーリストの親ノード

        Returns:

        """

        if(count == 0):
            return

        top_node = top_node.replace("<br>", "")
        # containerを取得
        query_result = self.data.get_container_query(self.selected_graph_name, \
                                                uri + top_node)

        if(len(query_result) > 0):
            self.node_list[top_node] = []
            self.node_id_list[top_node] = []
            self.object_list[top_node] = []
            self.object_id_list[top_node] = []
            self.edge_list[top_node] = []
            self.edge_id_list[top_node] = []
            self.treeview_item_list[top_node] = []
            count -= 1
            for result in query_result:
                container = result["source"]["value"]

            # Containerのメンバーを取得
            query_result = self.data.get_container_member_query(
                            self.selected_graph_name, container, uri)

            for result in query_result:
                value = result["value"]["value"]
                id = result["id"]["value"]
                self.node_list[top_node].append(value)
                self.node_id_list[top_node].append(id)

            # Containerにあるedgeを取得
            edge_results = self.data.get_container_edge_query(
                            self.selected_graph_name, container, uri)

            for edge in edge_results:
                edge_value = edge["value"]["value"]
                edge_id = edge["id"]["value"]
                self.edge_list[top_node].append(edge_value)
                self.edge_id_list[top_node].append(edge_id)

            # Containerにあるオブジェクトを取得
            query_object_result = self.data.get_obejct_in_container_query(
                                    self.selected_graph_name, container, uri)

            for obejct_result in query_object_result:
                object_id = obejct_result["id"]["value"]
                object_value = obejct_result["value"]["value"]
                self.object_list[top_node].append(object_value)
                self.object_id_list[top_node].append(object_id)

            for index, node in enumerate(self.node_list[top_node]):
                child = self.treeview.insert(
                    parent_node, tk.END, text=node, open=False)
                self.treeview_item_list[top_node].append(child)

                self.get_member_node_list(
                    uri, self.node_id_list[top_node][index], count, child)
        else:
            return

    def get_uri_list(self, node_id_list):
        """ top levelノードのuri listを取得

        Args:
            node_id_list: トップノードのidリスト

        Returns:

        """
        self.top_node_list_uri = {}
        self.con_ep_list = []
        for node_id in node_id_list["top"]:
            index = 0

            # nodeのuriを取得
            query_result = self.data.get_node_subject_id_query(
                            self.selected_graph_name, node_id)

            for result in query_result:
                temp_subject = result["s"]["value"]
                id = result["id"]["value"]
                node_subject = temp_subject.replace(id, "")
                if(node_subject not in self.top_node_list_uri):
                    self.top_node_list_uri[node_subject] = []
                    self.listbox_gpm_uri_list.insert(index, node_subject)
                    # container ep listに追加
                    self.con_ep_list.append(node_subject)
                    index += 1
                # コンテナ関連付けする時、新URIにidをコピーするため
                if(node_id not in self.top_node_list_uri[node_subject]):
                    self.top_node_list_uri[node_subject].append(node_id)

    def insert_lld_list(self):
        """ lld検索結果をリストに表示

        """
        self.listbox_lld.delete(0, tk.END)
        lld_action_list = self.fetch_lld_action(
            app.selected_graph, self.value_info.get("1.0", tk.END).rstrip('\r\n'))
        self.listbox_lld.insert(tk.END, *lld_action_list)

        if len(lld_action_list)==0:
            messagebox.showinfo("確認", "検索結果はありません！", parent=self)
        else:
            messagebox.showinfo("確認", "検索が完了しました！", parent=self)

    def insert_document_list(self):
        """ document検索結果をリストに表示

        """
        self.listbox_document.delete(0, tk.END)
        document_list = self.fetch_document(
            app.selected_graph, self.value_info.get("1.0", tk.END).rstrip('\r\n'))
        self.listbox_document.insert(tk.END, *document_list)

        if len(document_list)==0:
            messagebox.showinfo("確認", "検索結果はありません！", parent=self)
        else:
            messagebox.showinfo("確認", "検索が完了しました！", parent=self)

    def insert_engineer_list(self):
        """ engineer検索結果をリストに表示

        """
        self.listbox_engineer.delete(0, tk.END)
        engineer_list = self.fetch_engineer(
            app.selected_graph, self.value_info.get("1.0", tk.END).rstrip('\r\n'))
        self.listbox_engineer.insert(tk.END, *engineer_list)

        if len(engineer_list)==0:
            messagebox.showinfo("確認", "検索結果はありません！", parent=self)
        else:
            messagebox.showinfo("確認", "検索が完了しました！", parent=self)

    def insert_samfactor_list(self):
        """ sam因子検索結果をリストに表示

        """
        self.listbox_samfactor.delete(0, tk.END)

        if self.book_action.select():
            #idx = self.book_action.index('current')
            current_tab = self.book_action.tab(self.book_action.select(), "text")

            if current_tab == "Action":
                search_word = self.value_info.get("1.0", tk.END).rstrip('\r\n')
            elif current_tab == "Edge":
                selected_edge_index = self.listbox_edge.curselection()
                if(len(selected_edge_index) == 0):
                    messagebox.showerror("Error", "No edge data selected!")
                else:
                    search_word = self.listbox_edge.get(selected_edge_index)
        else:
            messagebox.showerror("Error", "No search data selected!")

        samfactor_list = self.get_samfactor_data(
            self.selected_graph_name, search_word)
        self.listbox_samfactor.insert(tk.END, *samfactor_list)

        if len(samfactor_list)==0:
            messagebox.showinfo("確認", "検索結果はありません！", parent=self)
        else:
            messagebox.showinfo("確認", "検索が完了しました！", parent=self)

    def fetch_lld_action(self, model, action_name):
        """ 過去事例検索

        Args:
            model:
            action_name:

        Returns:
            lld_action_list: 過去事例検索結果

        """
        query = """
        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
        PREFIX d3aki: <http://DigitalTriplet.net/2021/11/ontology/akiyama#>
        PREFIX d3: <http://digital-triplet.net/>
        PREFIX dcterms: <http://purl.org/dc/terms/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        select ?log ?log_action ?log_action_name (COUNT(?event) as ?count)
        where {
            {GRAPH <"""+model+""">
            {
                ?s pd3:value ?action_name.
                filter(?action_name = '"""+action_name+"""')
            }
            GRAPH ?log
            {
                ?log_ep pd3:epType "lld".
                ?log_action rdfs:seeAlso ?s;
                pd3:value ?log_action_name.
            }
            GRAPH <http://localhost:3030/akiyama/data/event>
            {
                OPTIONAL{?event d3aki:eventType "reference";
                        d3aki:referTo ?log_action}.}
            }
            UNION
            {
                GRAPH <"""+model+""">
            {
                ?s pd3:value ?action_name.
                filter(?action_name = '""" + action_name+"""')
            }
            GRAPH ?log
            {
                ?log_ep pd3:epType "lld".
                ?log_action rdfs:seeAlso ?s;
                pd3:value ?log_action_name.
            }
            }
            }
            group by ?log ?log_action ?log_action_name 
            order by DESC(?count)
        """
        results = self.fuseki.get_fuseki_data_json(query)
        lld_action_list = []
        for result in results:
            lld_action_list.append(result["log_action_name"]["value"])
        return lld_action_list

    def fetch_document(self, model, action_name):
        """ 知識文書検索

        Args:
            model:
            action_name:

        Returns:
            document_list: 知識文書検索結果

        """
        query = """
        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
        PREFIX d3aki: <http://DigitalTriplet.net/2021/11/ontology/akiyama#>
        PREFIX d3: <http://digital-triplet.net/>
        PREFIX dcterms: <http://purl.org/dc/terms/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        select ?documentTitle ?documentLink (COUNT(?event) as ?count)
        where {
        {
            GRAPH <"""+model+""">
            {
                ?s pd3:value ?action_name.
                filter(?action_name = '"""+action_name+"""')
            }
            GRAPH ?log
            {
                ?log_ep pd3:epType "lld".
                ?log_action rdfs:seeAlso ?s;
                pd3:value ?log_action_name.
                    ?document d3aki:reference ?log_action;
                            d3aki:documentURI ?documentURI.
            }
            GRAPH <http://localhost:3030/akiyama/data/document>
            {
                OPTIONAL{?documentURI d3aki:linkTo ?documentLink;
                                        d3aki:title ?documentTitle}.
            }
            GRAPH <http://localhost:3030/akiyama/data/event>
            {
                OPTIONAL{?event d3aki:eventType "reference";
                                d3aki:referTo ?documentURI;
                                d3aki:referedTo ?log_action}.
            }
        }
        UNION
        {
            GRAPH <"""+model+""">
            {
                ?s pd3:value ?action_name.
                filter(?action_name = '"""+action_name+"""')
            }
            GRAPH ?log
            {
                ?log_ep pd3:epType "lld".
                ?log_action rdfs:seeAlso ?s;
                pd3:value ?log_action_name.
                    ?document d3aki:reference ?log_action;
                            d3aki:documentURI ?documentURI.
            }
            GRAPH <http://localhost:3030/akiyama/data/document>
            {
                OPTIONAL{?documentURI d3aki:linkTo ?documentLink;
                                        d3aki:title ?documentTitle}.
            }
        }
        }
        group by ?documentTitle ?documentLink
        order by DESC(?count)
        """
        results = self.fuseki.get_fuseki_data_json(query)
        document_list = []
        for result in results:
            document_list.append(result["documentTitle"]["value"])
        return document_list

    def fetch_engineer(self, model, action_name):
        """ 技術者検索

        Args:
            model:
            action_name:

        Returns:
            engineer_list: 技術者検索結果

        """
        query = """
        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
        PREFIX d3aki: <http://DigitalTriplet.net/2021/11/ontology/akiyama#>
        PREFIX d3: <http://digital-triplet.net/>
        PREFIX dcterms: <http://purl.org/dc/terms/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
        select ?log ?log_action ?log_action_name ?engineer ?engineerName ?engineerLink (COUNT(?event) as ?count)
        where {
            {
            GRAPH <"""+model+""">
            {
                ?s pd3:value ?action_name.
                filter(?action_name = '"""+action_name+"""')
            }
            GRAPH ?log
            {
                ?log_ep pd3:epType "lld".
                ?log_action rdfs:seeAlso ?s;
                pd3:value ?log_action_name.
                ?engineer d3aki:practitioner ?log_action;
                            d3aki:engineerURI ?engineerURI;
                            pd3:value ?engineerName.
            }
            GRAPH <http://localhost:3030/akiyama/data/engineer>
            {
                OPTIONAL{?engineerURI d3aki:linkTo ?engineerLink;
                                    }.
            }
            GRAPH <http://localhost:3030/akiyama/data/event>
            {
                OPTIONAL{?event d3aki:eventType "reference";
                                d3aki:referTo ?engineerURI;
                                d3aki:referedTo ?log_action}.
            }
            }
            UNION
            {
            GRAPH <"""+model+""">
            {
                ?s pd3:value ?action_name.
                filter(?action_name = '"""+action_name+"""')
            }
            GRAPH ?log
            {
                ?log_ep pd3:epType "lld".
                ?log_action rdfs:seeAlso ?s;
                pd3:value ?log_action_name.
                ?engineer d3aki:practitioner ?log_action;
                            d3aki:engineerURI ?engineerURI;
                            pd3:value ?engineerName.
            }
            GRAPH <http://localhost:3030/akiyama/data/engineer>
            {
                OPTIONAL{?engineerURI d3aki:linkTo ?engineerLink;
                                    }.
            }
            }
        }
        group by ?log ?log_action ?log_action_name ?engineer ?engineerName ?engineerLink
        order by DESC(?count)
        """
        results = self.fuseki.get_fuseki_data_json(query)
        engineer_list = []
        for result in results:
            engineer_list.append(result["engineerName"]["value"])
        return engineer_list

    def get_samfactor_data(self, graph_name, action_name):

        samfactor_list = []

        if("#" in action_name):
            key_word = re.search('#(.+?)#', action_name).group(1)

        if('key_word' in locals()):
            sam_result = self.sam.query("MATCH (a{`分類` : '%s'})-[r]->(b) "
                                    "RETURN a.`分類` , properties(r) AS Type, b.`分類`" %(key_word))
            if sam_result:
                for sam_record in sam_result:
                    type = sam_record["Type"]
                    key = list(type.keys())[0]
                    samfactor = key_word + "-" + key + " " + "(" + str(type[key]) +")" + " " + \
                        "->" + sam_record["b.`分類`"]

                    if not samfactor in samfactor_list:
                        samfactor_list.append(samfactor)

        return samfactor_list

    def select_graph(self, event):
        """ グラフを選択処理

        Args:
            event: リスト選択イベント

        """
        w = event.widget
        select_index = int(w.curselection()[0])
        self.selected_graph = self.graph_url_list[select_index]

        self.selected_graph_name = "<" + self.selected_graph + ">"

        # treeviewを更新のため、元のtreeviewを削除
        if(self.treeview.get_children):
            for item in self.treeview.get_children():
                self.treeview.delete(item)

        # gpm listをクリア
        self.listbox_gpm_uri_list.delete(0, tk.END)

        # ノード情報をクリア
        self.clear_node_info()

        # オブジェクト表示リストをクリア
        self.listbox_object.delete(0, tk.END)

        # 知識文書、技術者、過去事例リストをクリアする
        self.listbox_document.delete(0, tk.END)
        self.listbox_engineer.delete(0, tk.END)
        self.listbox_lld.delete(0, tk.END)

        # SAM因子リストをクリアする
        self.listbox_samfactor.delete(0, tk.END)

        # エッジリストをクリア
        self.listbox_edge.delete(0, tk.END)

        (self.node_id_list) = self.data.get_top_node_list_in_graph(self.selected_graph_name)

        # Top level EP listを表示
        self.get_uri_list(self.node_id_list)

        # グラフを表示のため、ttlファイルを取得
        self.fuseki.get_graph_ttlfile(self.selected_graph_name, self.ttl_file)

    def tree_select(self, event):
        """ treeviewを選択処理

        Args:
            event: ツリーリスト選択イベント

        """
        # node名を取得
        record_id = self.treeview.focus()
        current_item = self.treeview.item(record_id)["text"]

        # node idを取得
        for key in self.treeview_item_list.keys():
            for index, item in enumerate(self.treeview_item_list[key]):
                if(item == record_id):
                    node_id = self.node_id_list[key][index]
                    new_graph_level = key

        #エッジidを取得
        edge_id = None
        if(new_graph_level == self.graph_level):
            if self.select_edge_index != None:
                edge_id = self.edge_id_list[self.graph_level][self.select_edge_index]

        # ノード情報表示をクリア
        self.clear_node_info()

        # オブジェクト表示リストをクリア
        self.listbox_object.delete(0, tk.END)

        # ノード情報を取得(id, actionType, layer, position など)
        (id, action, layer) = self.data.get_node_info(
                                self.selected_graph_name, node_id)

        (use, useby) = self.data.get_use_info(self.selected_graph_name, \
                                            self.gpm_uri, node_id)

        # ActionInput, ActionOutputを取得
        (actionInput, actionOutput) = self.data.get_inputOutput_info(
                                    self.selected_graph_name, self.gpm_uri, \
                                    node_id)

        # ノード情報を表示
        self.node_info_update(self.id_info, id)
        self.node_info_update(self.action_info, action)
        self.node_info_update(self.layer_info, layer)
        self.node_info_update(self.value_info, current_item)

        self.node_info_update(self.use_info, ", ".join(use))
        self.node_info_update(self.useby_info, ", ".join(useby))
        self.node_info_update(self.actionInput_info, ", ".join(actionInput))
        self.node_info_update(self.actionOutput_info, ", ".join(actionOutput))

        # クエリを投げるため、ノードの主語を取得
        target_node = self.data.get_node_subject_info(self.selected_graph_name, \
                                                    node_id)
        # 補足情報を取得と表示(intention, annotation, rationale, objectなど)
        self.set_supplement_info(target_node)

        image_path = "selected_graph.png"
        # 選択したノードの層と表示しているグラフの層が異なる場合、グラフとエッジリスト更新
        if(new_graph_level != self.graph_level):
            self.graph_level = new_graph_level

            (self.graph_node_list, self.graph_edge_list, \
            self.graph_label_list, self.graph_position) = \
                self.data.get_graph_data_by_ttlfile(self.gpm_uri, \
                            self.node_id_list[self.graph_level], \
                            self.object_id_list[self.graph_level], \
                            edge_id, self.ttl_file)

            # エッジリストをクリア
            self.listbox_edge.delete(0, tk.END)
            for edge in self.edge_list[self.graph_level]:
                # edge valueを設定されてない場合、NULLで表示する
                if(len(edge) == 0):
                    self.listbox_edge.insert(tk.END, "NULL")
                else:
                    self.listbox_edge.insert(tk.END, edge)

        # 画像を設定
        set_image(image_path, self.graph_node_list, self.graph_edge_list, \
                self.graph_label_list, [node_id], \
                self.graph_position)

        # 画像を更新
        selected_image = Image.open(image_path)
        self.change_image(selected_image)

    def select_edge(self, event):
        """ edge listを選択する処理

        """
        w = event.widget
        self.select_edge_index = int(w.curselection()[0])
        selected_edge = self.listbox_edge.get(self.select_edge_index)

        #エッジidを取得
        edge_id = self.edge_id_list[self.graph_level][self.select_edge_index]

        #オブジェクトリスト選択クリア
        self.listbox_object.select_clear(0, tk.END)

        #Action lists　選択ノード
        #node名を取得
        node_id = None
        record_id = self.treeview.focus()
        current_item = self.treeview.item(record_id)["text"]

        # node idを取得
        for key in self.treeview_item_list.keys():
            for index, item in enumerate(self.treeview_item_list[key]):
                if(item == record_id):
                    node_id = self.node_id_list[key][index]

        image_path = "selected_graph.png"

        (self.graph_node_list, self.graph_edge_list, \
            self.graph_label_list, self.graph_position) = \
            self.data.get_graph_data_by_ttlfile(self.gpm_uri, \
                self.node_id_list[self.graph_level], \
                self.object_id_list[self.graph_level], edge_id, self.ttl_file)

        # 画像を設定
        set_image(image_path, self.graph_node_list, self.graph_edge_list, \
                self.graph_label_list, [node_id], \
                self.graph_position)

        # 画像を更新
        selected_image = Image.open(image_path)
        self.change_image(selected_image)

    def clear_node_info(self):
        """ ノード情報表示をクリア処理

        """
        self.node_info_clear(self.id_info)
        self.node_info_clear(self.action_info)
        self.node_info_clear(self.layer_info)
        self.node_info_clear(self.value_info)
        self.node_info_clear(self.use_info)
        self.node_info_clear(self.useby_info)

        self.node_info_clear(self.intention_info)
        self.node_info_clear(self.rationale_info)
        self.node_info_clear(self.annotation_info)
        self.node_info_clear(self.tool_info)

        self.node_info_clear(self.actionInput_info)
        self.node_info_clear(self.actionOutput_info)

    def set_supplement_info(self, node):
        """ ノード補足情報を取得と表示処理

        Args:
            node: 選択したノード(uri + node id)

        """
        # ------------------------------------------------------------------
        # intentionを取得
        # ------------------------------------------------------------------
        (intention_id, intention_value) = self.data.get_supplement_detail_info(
            self.selected_graph_name, "intention", node)

        if(intention_id is not None) and (intention_value is not None):
            self.node_info_update(self.intention_info, intention_value)

        # ------------------------------------------------------------------
        # rationaleを取得
        # ------------------------------------------------------------------
        (rationale_id, rationale_value) = self.data.get_supplement_detail_info(
            self.selected_graph_name, "rationale", node)

        if(rationale_id is not None) and (rationale_value is not None):
            self.node_info_update(self.rationale_info, rationale_value)

        # ------------------------------------------------------------------
        # annotationを取得
        # ------------------------------------------------------------------
        (annotation_id, annotation_value) = self.data.get_supplement_detail_info(
            self.selected_graph_name, "annotation", node)

        if(annotation_id is not None) and (annotation_value is not None):
            self.node_info_update(self.annotation_info, annotation_value)

        # ------------------------------------------------------------------
        # toolを取得
        # ------------------------------------------------------------------
        (tool_id, tool_value) = self.data.get_supplement_detail_info(
            self.selected_graph_name, "tool/knowledge", node)

        if(tool_value is not None):
            self.node_info_update(self.tool_info, tool_value)

        # ------------------------------------------------------------------
        # objectを取得
        # ------------------------------------------------------------------
        (object_id, object_value) = self.data.get_supplement_detail_info(
            self.selected_graph_name, "", node)

        if(object_id is not None) and (object_value is not None):
            for index, item in enumerate(object_value[node]):
                self.listbox_object.insert(index, item)

    def change_image(self, img):
        """ グラフ画像の入り替え処理

        Args:
            img: 更新する画像

        """
        # canvasの書き換え
        self.graph_canvas.photo = ImageTk.PhotoImage(img)
        self.graph_canvas.itemconfig(
            self.image_on_canvas, image=self.graph_canvas.photo)

    def insert_edge_list(self):

        query = """
        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
        SELECT ?s ?value
        WHERE {
            graph """ + self.selected_graph_name + """ {
            ?s rdf:type pd3:Flow;
                pd3:value ?value.
            FILTER(regex(str(?s),""" + '"' + self.gpm_uri + '"' + """))
            }
        }
        """
        results = self.fuseki.get_fuseki_data_json(query)
        edge_value_list = []
        if(len(results) > 0):
            for result in results:
                if len(result["value"]["value"]) != 0:
                    edge_value_list.append(result["value"]["value"])

            self.listbox_edge.insert(tk.END, *edge_value_list)

    def show_top_graph(self):
        """ グラフのTOP部分を表示

        """
        # グラフ情報を取得
        (self.graph_node_list, self.graph_edge_list, \
            self.graph_label_list, self.graph_position) = \
            self.data.get_graph_data_by_ttlfile(self.gpm_uri, \
                        self.node_id_list["top"], \
                        self.object_id_list["top"], None, self.ttl_file)

        # 最上層グラフを表示
        image_path = "top_graph.png"
        self.graph_level = "top"
        set_image(image_path, self.graph_node_list,\
                self.graph_edge_list, self.graph_label_list, [], \
                self.graph_position)
        top_image = Image.open(image_path)

        self.graph_canvas.photo = ImageTk.PhotoImage(top_image)
        self.image_on_canvas = self.graph_canvas.create_image(
            -100, -100, anchor=tk.NW, image=self.graph_canvas.photo)

    def set_gpm(self):
        """ treeviewとグラフで確認したいGPMを設定

        """
        selected_GPM_index = self.listbox_gpm_uri_list.curselection()
        if(len(selected_GPM_index) == 0):
            messagebox.showerror("Error", "No data selected!")
        else:
            self.gpm_uri = self.listbox_gpm_uri_list.get(selected_GPM_index)

            # treeviewを更新のため、元のtreeviewを削除
            if(self.treeview.get_children):
                for item in self.treeview.get_children():
                    self.treeview.delete(item)

            # ノード情報をクリア
            self.clear_node_info()

            # オブジェクト表示リストをクリア
            self.listbox_object.delete(0, tk.END)

            # エッジリストをクリア
            self.listbox_edge.delete(0, tk.END)

            # treeviewを表示
            self.get_hierarchy_node_list(self.gpm_uri)

            # エッジリスト選択を初期化
            self.select_edge_index = None

            # エッジリストを表示
            for edge in self.edge_list["top"]:
                # edge valueを設定されてない場合、NULLで表示する
                if(len(edge) == 0):
                    self.listbox_edge.insert(tk.END, "NULL")
                else:
                    self.listbox_edge.insert(tk.END, edge)

            # グラフを表示
            self.show_top_graph()

    def delete_member_event(self):
        """ コンテナメンバー削除処理

        """
        # treeviewで選択したノードnode名とidを取得
        (current_item, current_node_id) = self.get_selected_node_id()

        #current_item = current_item.replace("<br>", "")

        if(current_item is not None) or (current_node_id is not None):
            # 選択したノードがコンテナを含まれてない場合、エラーメッセージ
            if not current_node_id in self.node_list.keys():
                messagebox.showerror(
                    "Error", "選択したアクションがメンバーはありません!", parent=self)
            # 選択したノードがコンテナを含まれている場合、メンバー削除処理
            else:
                # オブジェクトリストを取得
                object_id_list = self.object_id_list[current_node_id]

                # コピーしてから削除する
                now = datetime.datetime.now()
                new_uri = "http://localhost/Container_delete_" + \
                    now.strftime("%Y%m%d_%H%M%S") + "/"

                assoc.copy_ep_data(self.selected_graph_name, self.gpm_uri, \
                                    new_uri, self)

                assoc.delete_member(self.selected_graph_name, self.gpm_uri, 
                            new_uri, current_item, current_node_id, \
                            self.node_list, self.node_id_list, \
                            object_id_list, self)
                # ウィジェット更新
                self.update_after_member_modfiy()

                # 新URIプロセスをコピーしたため、グラフのttlファイルを再取得
                self.fuseki.get_graph_ttlfile(
                    self.selected_graph_name, self.ttl_file)

    def insert_use_event(self):
        """ use関連付け追加処理

        """
        # 選択したノード名とidを取得
        (current_item, current_node_id) = self.get_selected_node_id()

        if(current_item is not None) or (current_node_id is not None):
            # 選択したノードのuriを取得
            node_uri = self.gpm_uri

            log_graph = self.log_graph.get()
            log_graph_uri = self.log_uri.get()
            log_graph_node = self.log_node.get()

            if (not log_graph) and (not log_graph_uri) and (not log_graph_node):
                messagebox.showerror("Error", "ログ情報を選択していません!", parent=self)
            else:
                if(log_graph == self.selected_graph_name) and (log_graph_uri == node_uri):
                    messagebox.showerror(
                        "Error", "プロセス内use関連付けができません!", parent=self)
                else:
                    assoc.insert_use(self.selected_graph_name, node_uri, \
                            current_node_id, log_graph, log_graph_uri, log_graph_node)

                    # ノード情報表示を更新
                    (use, useby) = self.data.get_use_info(
                        self.selected_graph_name, node_uri, current_node_id)
                    self.node_info_clear(self.useby_info)
                    self.node_info_update(self.useby_info, ', '.join(useby))

    def delete_use_event(self):
        """ use関連付け削除処理

        """
        # 選択したノード名とidを取得
        (current_item, current_node_id) = self.get_selected_node_id()

        if(current_item is not None) or (current_node_id is not None):
            # 選択したノードのuriを取得
            node_uri = self.gpm_uri

            log_graph = self.log_graph.get()
            log_graph_uri = self.log_uri.get()
            log_graph_node = self.log_node.get()

            if (not log_graph) and (not log_graph_uri) and (not log_graph_node):
                messagebox.showerror("Error", "ログ情報を選択していません!", parent=self)
            else:
                if(log_graph == self.selected_graph_name) and (log_graph_uri == node_uri):
                    messagebox.showerror(
                        "Error", "プロセス内use関連付け削除ができません!", parent=self)
                else:
                    query = """
                        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                        SELECT distinct ?s ?id ?useby
                        WHERE {
                            graph """ + self.selected_graph_name + """ {
                            ?s pd3:id ?id;
                                pd3:useBy ?useby.
                            FILTER(?id = """ + '"' + current_node_id + '"' + """) 
                            FILTER(regex(str(?s),""" + '"' + node_uri + '"' + """))
                            }
                        }
                    """
                    results = self.fuseki.get_fuseki_data_json(query)
                    if(len(results) > 0):
                        useby_list = []
                        for result in results:
                            useby_list.append(result["useby"]["value"])
                        log_node = log_graph_uri + log_graph_node

                        # 削除nodeが存在している場合、削除する
                        if log_node in useby_list:
                            assoc.delete_use(self.selected_graph_name, node_uri, \
                                    current_node_id, log_graph, log_graph_uri, \
                                    log_graph_node)

                            # ノード情報表示を更新
                            (use, useby) = self.data.get_use_info(
                                self.selected_graph_name, self.gpm_uri, \
                                current_node_id)
                            self.node_info_clear(self.useby_info)
                            self.node_info_update(self.useby_info, ", ".join(useby))
                            
                        else:
                            messagebox.showinfo("確認", "削除する情報はありません!", parent=self)
                    else:
                        messagebox.showinfo("確認", "削除する情報はありません!", parent=self)

    def node_info_update(self, widget, data):
        """ ノード情報更新処理

        Args:
            widget: 更新するウェジット
            data: 更新するデータ

        """
        widget.configure(state="normal")
        widget.insert(1.0, data)
        widget.configure(state="disabled")

    def node_info_clear(self, widget):
        """ ノード情報クリア処理

        Args:
            widget: 更新するウェジット

        """
        widget.configure(state="normal")
        widget.delete("1.0", "end")
        widget.configure(state="disabled")

    def get_selected_node_id(self):
        """ treeviewで選択したノードのidを取得

        Returns:
            current_item: ノード名
            current_node_id: ノードid

        """
        current_item = None
        current_node_id = None
        # ノードnode名を取得
        record_id = self.treeview.focus()
        if(not record_id):
            messagebox.showerror("Error", "No node selected!")
        else:
            current_item = self.treeview.item(record_id)["text"]

            # ノードidを取得
            for key in self.treeview_item_list.keys():
                for index, item in enumerate(self.treeview_item_list[key]):
                    if(record_id == item):
                        current_node_id = self.node_id_list[key][index]

        return current_item, current_node_id

    def update_after_member_modfiy(self):
        """ メンバー修正後更新処理

        Top level ep listを更新, treeview, ノード情報, container ep list, 
        canvasをクリア

        """
        # treeviewを更新のため、元のtreeviewを削除
        if(self.treeview.get_children):
            for item in self.treeview.get_children():
                self.treeview.delete(item)

        # gpm listの選択をクリア
        self.listbox_gpm_uri_list.delete(0, tk.END)

        # ノード情報をクリア
        self.clear_node_info()

        # エッジリストをクリア
        self.listbox_edge.delete(0, tk.END)

        # オブジェクト表示リストをクリア
        self.listbox_object.delete(0, tk.END)

        (self.node_id_list) = self.data.get_top_node_list_in_graph(
                                self.selected_graph_name)

        self.get_uri_list(self.node_id_list)

        # canvasクリア
        self.graph_canvas.delete("all")

    def select_object(self, event):
        """ object id listを選択処理

        Args:
            event: リスト選択イベント

        """
        w = event.widget
        select_object_index = int(w.curselection()[0])
        object_value = self.listbox_object.get(select_object_index)

        for key in self.object_list.keys():
            for index, value in enumerate(self.object_list[key]):
                if(value == object_value):
                    object_id = self.object_id_list[key][index]

        # グラフ表示
        image_path = "selected_graph.png"

        (self.graph_node_list, self.graph_edge_list, \
            self.graph_label_list, self.graph_position) = \
            self.data.get_graph_data_by_ttlfile(self.gpm_uri, \
                        self.node_id_list[self.graph_level], \
                        self.object_id_list[self.graph_level], \
                        None, self.ttl_file)

        # 画像を設定
        set_image(image_path, self.graph_node_list, self.graph_edge_list, \
                self.graph_label_list, [object_id], \
                self.graph_position)

        # 画像を更新
        selected_image = Image.open(image_path)
        self.change_image(selected_image)

    def insert_ep_member_event(self):
        """ EPをコンテナとして埋め込み処理

        """
        # treeviewで選択したノード名とidを取得
        (current_item, current_node_id) = self.get_selected_node_id()

        if(current_item is not None) or (current_node_id is not None):
            # treeviewで選択したノードのuriを取得
            node_uri = self.gpm_uri

            member_graph = self.win2_graph.get()
            member_uri = self.win2_uri.get()

            if (not member_graph) and (not member_uri):
                messagebox.showerror("Error", "プロセス情報を選択していません!", parent=self)
            else:
                if (self.selected_graph_name == member_graph) and \
                    (node_uri == member_uri):
                    messagebox.showerror(
                        "Error", "同じプロセスをコンテナのメンバー関連付けができません!", parent=self)
                else:
                    # ----------------------------------------------------
                    # 新uriで関連付けする両プロセスをコピーし、
                    # 新uriのデータでコンテナ関連付けする
                    # ----------------------------------------------------
                    # 新uriを生成する
                    now = datetime.datetime.now()
                    new_uri = "http://localhost/Container_add_ep" + \
                            now.strftime("%Y%m%d_%H%M%S") + "/"
                    new_prefix = "GPM_Container_" + now.strftime("%Y%m%d_%H%M%S")

                    assoc.copy_ep_data(self.selected_graph_name, node_uri, new_uri, self)
                    assoc.copy_ep_data(member_graph, member_uri, new_uri, self)

                    assoc.insert_member_new(self.selected_graph_name, \
                                    current_item, current_node_id, \
                                    member_graph, member_uri, \
                                    new_uri, self)

                    # ウィジェット更新
                    self.update_after_member_modfiy()

                    # 新URIプロセスをコピーしたため、グラフのttlファイルを再取得
                    self.fuseki.get_graph_ttlfile(
                        self.selected_graph_name, self.ttl_file)

    def insert_con_member_event(self):
        """ コンテナが埋め込み処理

        """
        # treeviewで選択したノード名とidを取得
        (current_item, current_node_id) = self.get_selected_node_id()

        if(current_item is not None) or (current_node_id is not None):
            # treeviewで選択したノードのuriを取得
            node_uri = self.gpm_uri

            member_graph = self.win2_graph.get()
            member_graph_uri = self.win2_uri.get()
            member_graph_node = self.win2_selected_item.get()

            if (not member_graph) and (not member_graph_uri):
                messagebox.showerror("Error", "プロセス情報を選択していません!", parent=self)
            else:
                if not member_graph_node:
                    messagebox.showerror(
                        "Error", "コンテナを選択していません!", parent=self)
                else:
                    # コンテナであるかチェック
                    check_result = self.data.get_container_query(
                                    member_graph, member_graph_uri + \
                                    member_graph_node)

                    if(len(check_result) > 0):
                        # 選択したノードはコンテナではある
                        for result in check_result:
                            selected_container = result["source"]["value"]

                        # コンテナのメンバーを取得
                        (node_list, id_list) = self.data.get_container_member(
                            member_graph, member_graph_uri, selected_container)

                        # ----------------------------------------------------
                        # 新uriで関連付けするプロセスとコンテナをコピーし、
                        # 新uriのデータでコンテナ関連付けする
                        # ----------------------------------------------------
                        # 新uriを生成する
                        now = datetime.datetime.now()
                        new_uri = "http://localhost/Container_add_" + \
                                now.strftime("%Y%m%d_%H%M%S") + "/"
                        new_prefix = "GPM_Container_" + now.strftime("%Y%m%d_%H%M%S")

                        assoc.copy_ep_data(self.selected_graph_name, node_uri, new_uri, self)

                        assoc.copy_container_data(member_graph, member_graph_uri, \
                            selected_container, new_uri, id_list, self)

                        container_id = selected_container.replace(member_graph_uri, "")

                        # コンテナメンバー関連付け
                        assoc.insert_con_member_new(self.selected_graph_name, \
                                        current_item, current_node_id, \
                                        new_uri, container_id, self)

                        # ウィジェット更新
                        self.update_after_member_modfiy()

                        # アクション名を変更のため、グラフのttlファイルを再取得
                        self.fuseki.get_graph_ttlfile(
                            self.selected_graph_name, self.ttl_file)
                    else:
                        # 選択したノードはコンテナではない
                        messagebox.showerror(
                            "Error", "選択したのはコンテナではありません!", parent=self)

    def open_log_window(self):
        """ ログウインドウを開く

        """
        logWindow(self.log_graph, self.log_uri, self.log_node, self)

    def open_window_2(self):
        """ ウインドウ2を開く

        """
        window_2(self.win2_graph, self.win2_uri, self.win2_selected_item, self)

    def targetItem(self, event):
        """ 指定されたアイテムを反映

        Args:
            event: イベント

        """
        self.iid = self.tree.focus()
        if self.iid:
            self.parentEntity.set(self.tree.item(self.iid, "text"))

            # prop listを表示
            self.propNameList = []
            self.propList.delete(0, tk.END)
            if self.iid in self.propInfo.keys():
                for prop in self.propInfo[self.iid].keys():
                    self.propNameList.append(prop)
            self.listboxPropValue.set(self.propNameList)

            # prop name入力内容をクリア
            self.addPropNameEntry.delete(0, tk.END)  
            # prop value入力内容をクリア
            self.addPropValueEntry.delete(0, tk.END) 

    def readRDFfile(self, filePath):
        """ ttlフィアルを読み込み

        Args:
            filePath: ttlファイルパス

        """
        # ルート要素のみの場合、取得のため
        dataTempList = {}
        obejctList = {}

        pd3 = Namespace('http://DigitalTriplet.net/2021/08/ontology#')

        with open(filePath, 'r', encoding='utf-8') as f:
            g = Graph()
            g.parse(f)

        for s,p,o in g.triples((None, RDF.type, pd3.Object)):
            # ルート要素は子要素がある場合
            for s1,p1,o1 in g.triples((s, pd3.partOf, None)):
                for s2,p2,o2 in g.triples((s, pd3.value, None)): 
                    child = str(o2)
                    childUri = str(s2)
                    for s3,p3,o3 in g.triples((o1, None, None)):
                        if p3 == pd3.value:
                            root = str(o3)
                            if root not in self.treeUriList["top"].keys():
                                self.treeUriList["top"][root] = str(o1)
                                dataTempList[root] = str(o1)
                                self.treeUriList[root] = {}
                    self.treeUriList[root][child] = childUri
                    dataTempList[child] = childUri

        # ルート要素は子要素がない場合
        for s,p,o in g.triples((None, RDF.type, pd3.Object)):
            for s1,p1,o1 in g.triples((s, pd3.value, None)): 
                obejctList[str(o1)] = str(s)

        for data in obejctList.keys():
            # ルート要素を追加
            if data not in dataTempList.keys():
                self.treeUriList["top"][data] = obejctList[data]

        # treeviewを作成
        for root in self.treeUriList["top"]:
            self.rootiid = self.tree.insert("", "end", text=root)
            self.entityTree["top"].append(self.rootiid)
            #treeIdTable[root] = self.rootiid
            self.treeIdTable['top'][root] = self.rootiid
            if root in self.treeUriList.keys():
                if root not in self.treeIdTable.keys():
                    self.treeIdTable[root] = {}
                for item in self.treeUriList[root]:
                    insert_iid = self.tree.insert(self.rootiid, "end", text=item)
                    if self.rootiid not in self.entityTree.keys():
                        self.entityTree[self.rootiid] = []
                    self.entityTree[self.rootiid].append(insert_iid)
                    #treeIdTable[item] = insert_iid
                    self.treeIdTable[root][item] = insert_iid

        # parametersを修正
        for s,p,o in g.triples((None, pd3.hasParameter, None)):
            parentValue = None
            for s4,p4,o4 in g.triples((s, pd3.partOf, None)):
                for s5,p5,o5 in g.triples((o4, pd3.value, None)):
                    parentValue = str(o5)
            for s1,p1,o1 in g.triples((s, pd3.value, None)): 
                target = str(o1)
            if parentValue is None:
                targetId = self.treeIdTable['top'][target]
            else:
                targetId = self.treeIdTable[parentValue][target]
  
            for s2,p2,o2 in g.triples((o, pd3.paramName, None)): 
                propName = str(o2)
            for s3,p3,o3 in g.triples((o, pd3.paramValue, None)): 
                paramValue = str(o3)

            if targetId not in self.propInfo.keys():
                self.propInfo[targetId] = {}
                self.propInfoUri[targetId] = {}
            if 'propName' in locals() and 'paramValue' in locals():
                self.propInfo[targetId][propName] = paramValue
                self.propInfoUri[targetId][propName] = str(o)

    def insertRootEntity(self):
        """ ルート要素を追加処理

        """
        if len(self.rootEntity.get()) == 0:
            messagebox.showerror("Error", "入力してください!")

        insertData = self.rootEntity.get()

        # 唯一名を生成
        id = "".join(random.choice(
            string.ascii_letters + string.digits) for _ in range(20)) + "-1"

        # fusekiに同期する
        self.fusekiInsert.append([URIRef(self.entityUri+id), \
                    RDF.type, self.pd3.Object])
        self.fusekiInsert.append([URIRef(self.entityUri+id), \
                    self.pd3.value, Literal(insertData)])

        if self.fuseki.insert(self.entityGraph, self.fusekiInsert) is False:
            messagebox.showerror("Error", "Fusekiにインサート失敗!")
        else:
            # rootのiidを登録
            insertData = self.rootEntity.get()
            self.rootiid = self.tree.insert("", "end", text= insertData)
            self.iid = self.rootiid
            self.entityTree["top"].append(self.rootiid)
            self.treeUriList["top"][insertData] = self.entityUri+id
            self.treeIdTable["top"][insertData] = self.rootiid
            # 追加したらEntryの値を削除する
            self.rootEntity.set("")

        self.fusekiInsert.clear()

    def deleteEntity(self):
        """ 削除処理

        """
        relationAll = {}

        # 先にFuseki entity情報削除
        parent_iid = self.tree.parent(self.iid)
        delete_text = self.tree.item(self.iid, "text") 

        # root entityの削除ではない場合
        if len(parent_iid) != 0:
            deleteQuery =  """
                PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                WITH """ + self.entityGraph + """
                DELETE { ?s ?p ?o }
                WHERE { ?s ?p ?o;
                        pd3:value ?value.
                        FILTER(?value = """ + '"' + delete_text + '"' + """)}""" 

            if self.fuseki.update(deleteQuery):
                # propertyがある場合
                if self.iid in self.propInfoUri.keys():
                    for prop in self.propInfoUri[self.iid]:
                        uri = self.propInfoUri[self.iid][prop]

                        # parametersと関連するrelationshipを取得
                        relation = self.GetRelationByParam(uri)
                        if uri not in relationAll.keys():
                            relationAll[uri] = []
                        for data in relation:
                            relationAll[uri].append(data)

                        deletePropValueQuery = """
                            PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                            WITH """ + self.entityGraph + """
                            DELETE { ?s ?p ?o }
                            WHERE { ?s ?p ?o.
                                    FILTER(?s = """ + '<' + uri + '>' + """)}""" 

                        if self.fuseki.update(deletePropValueQuery) is False:
                            messagebox.showerror("Error", "プロパティ削除が失敗しました!")

                # treeUriList と treeIdTable削除
                parent_text = self.tree.item(parent_iid, "text")
                del self.treeUriList[parent_text][delete_text]
                del self.treeIdTable[parent_text][delete_text]

                messagebox.showinfo("確認", "削除しました。")

        # root entityを削除の場合
        else:
            parent_text = None
            rootUri = self.treeUriList["top"][delete_text]

            deleteEntityQuery =  """
                PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                WITH """ + self.entityGraph + """
                DELETE { ?s ?p ?o }
                WHERE { ?s ?p ?o.
                        FILTER(?s = """ + '<' + rootUri + '>' + """)}""" 

            if self.fuseki.update(deleteEntityQuery) is False:
                messagebox.showerror("Error", "Root Entity削除が失敗しました!")

            # root entityはpropertyがある場合削除する
            if self.iid in self.propInfoUri.keys():
                for prop in self.propInfoUri[self.iid]:
                    uri = self.propInfoUri[self.iid][prop]

                    # parametersと関連するrelationshipを取得
                    relation = self.GetRelationByParam(uri)
                    if uri not in relationAll.keys():
                        relationAll[uri] = []
                    for data in relation:
                        relationAll[uri].append(data)

                    deletePropValueQuery = """
                        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                        WITH """ + self.entityGraph + """
                        DELETE { ?s ?p ?o }
                        WHERE { ?s ?p ?o.
                                FILTER(?s = """ + '<' + uri + '>' + """)}""" 

                    if self.fuseki.update(deletePropValueQuery) is False:
                        messagebox.showerror("Error", "プロパティ削除が失敗しました!")

            # 子要素を削除
            if delete_text in self.treeUriList.keys():
                for childName, childeUri in self.treeUriList[delete_text].items():
                    deleteQuery = """
                        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                        WITH """ + self.entityGraph + """
                        DELETE { ?s ?p ?o }
                        WHERE { ?s ?p ?o.
                                FILTER(?s = """ + '<' + childeUri + '>' + """)}""" 
                    if (self.fuseki.update(deleteQuery)) is False:
                        messagebox.showerror("Error", "子要素削除が失敗しました!")
    
                # 子要素にpropertyがある場合
                for name, childeIid in self.treeIdTable[delete_text].items():
                    if childeIid in self.propInfoUri.keys():
                        for prop in self.propInfoUri[childeIid]:
                            uri = self.propInfoUri[childeIid][prop]

                            # parametersと関連するrelationshipを取得
                            relation = self.GetRelationByParam(uri)
                            if uri not in relationAll.keys():
                                relationAll[uri] = []
                            for data in relation:
                                relationAll[uri].append(data)

                            deletePropValueQuery = """
                                PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                                WITH """ + self.entityGraph + """
                                DELETE { ?s ?p ?o }
                                WHERE { ?s ?p ?o.
                                        FILTER(?s = """ + '<' + uri + '>' + """)}""" 

                            if self.fuseki.update(deletePropValueQuery) is False:
                                messagebox.showerror("Error", "プロパティ削除が失敗しました!")

            # treeUriList と treeIdTable削除
            del self.treeUriList["top"][delete_text]
            if delete_text in self.treeUriList.keys():
                del self.treeUriList[delete_text]
            del self.treeIdTable["top"][delete_text]
            if delete_text in self.treeIdTable.keys():
                del self.treeIdTable[delete_text]

            messagebox.showinfo("確認", "削除しました。")

        # relation param削除
        for paramKey in relationAll.keys():
            for data in relationAll[paramKey]:
                delRelationQuery = """
                    PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                    WITH """ + self.entityGraph + """
                    DELETE { ?s pd3:include """ + '<' + paramKey + '>' + """ }
                    WHERE { ?s ?p ?o.
                            FILTER(?s = """ + '<' + data + '>' + """)}""" 

                if self.fuseki.update(delRelationQuery) is False:
                    messagebox.showerror("Error", "Fuseki削除失敗!")
                else:
                    # relation param表示リスト更新
                    for relationName, relationUri in self.relationUri.items():
                        if relationUri == data:
                            for index, param in enumerate(self.relationInfoListUri[relationName]):
                                if param == paramKey:
                                    del self.relationInfoList[relationName][index]
                                    del self.relationInfoListUri[relationName][index]
        # relation param 表示を更新する
        relationIndex = self.relationList.curselection()
        selectedRelation = self.relationList.get(relationIndex)
        self.listboxRelationParams.set("")
        self.listboxRelationParams.set(self.relationInfoList[selectedRelation])
            
        self.tree.delete(self.iid)
        if self.iid in self.entityTree.keys():
            del self.entityTree[self.iid]

        for subject in self.entityTree.keys():
            for item in self.entityTree[subject]:
                if self.iid == item:
                    self.entityTree[subject].remove(item)
    
        # protperty情報があれば削除する
        if self.iid in self.propInfo.keys():
            del self.propInfo[self.iid]
            # prop表示リストを削除する
            self.propList.delete(0,tk.END)
        
        # Target Entityを削除する
        self.parentEntity.set("")

    def editEntity(self):
        """ 編集処理

        """
        newValue = self.parentEntity.get()
        editValue = self.tree.item(self.iid, "text")

        # fusekiに同期する
        updateQuery = """
            PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
            WITH """ + self.entityGraph + """
            DELETE { ?s pd3:value """ + '"' + editValue + '"' + """ }
            INSERT { ?s pd3:value """ + '"' + newValue + '"' + """ }
            WHERE { ?s ?p ?o;
                      pd3:value """ + '"' + editValue + '"' + """. }""" 

        if self.fuseki.update(updateQuery) is False:
            messagebox.showerror("Error", "Fusekiにインサート失敗!")
        else:
            # treeveiwを更新
            self.tree.item(self.iid, text=self.parentEntity.get())

            parent_iid = self.tree.parent(self.iid)
            # root entryではない場合
            if len(parent_iid) != 0:
                parent_entity = self.tree.item(parent_iid, "text")
                uri = self.treeUriList[parent_entity][editValue]

                # treeUriList更新する
                del self.treeUriList[parent_entity][editValue]
                self.treeUriList[parent_entity][newValue] = uri
                # treeIdTable更新する
                del self.treeIdTable[parent_entity][editValue]
                self.treeIdTable[parent_entity][newValue] = self.iid
                # self.entityTree更新する
                for index, item in enumerate(self.entityTree[parent_iid]):
                    if item == self.iid:
                        del self.entityTree[parent_iid][index]
                        self.entityTree[parent_iid].append(self.iid)

                # relationの情報を更新
                for data in self.relationInfoList.values():
                    for index, item in enumerate(data):
                        check1 = parent_entity + "->" + editValue + "->"
                        check2 = parent_entity + "->" + editValue + "."
                        temp = item
                        if check1 in item:
                            updateValue = parent_entity + "->" + newValue + "->"
                            # 古い値を削除
                            data.pop(index)
                            # 編集値を更新
                            update = temp.replace(check1, updateValue)
                            data.insert(index, update)
                        elif check2 in item:
                            updateValue = parent_entity + "->" + newValue + "."
                            # 古い値を削除
                            data.pop(index)
                            # 編集値を更新
                            update = temp.replace(check2, updateValue)
                            data.insert(index, update)
            # root entryの場合
            else:
                uri = self.treeUriList["top"][editValue]

                # treeUriList更新する
                del self.treeUriList["top"][editValue]
                self.treeUriList["top"][newValue] = uri
                # 子要素がある場合
                if editValue in self.treeUriList.keys():
                    self.treeUriList[newValue] = self.treeUriList[editValue]
                    del self.treeUriList[editValue]
                # treeIdTable更新する
                del self.treeIdTable["top"][editValue]
                self.treeIdTable["top"][newValue] = self.iid
                # 子要素がある場合
                if editValue in self.treeIdTable.keys():
                    self.treeIdTable[newValue] = self.treeIdTable[editValue]
                    del self.treeIdTable[editValue]
                # self.entityTree更新する
                for index, item in enumerate(self.entityTree["top"]):
                    if item == self.iid:
                        del self.entityTree["top"][index]
                        self.entityTree["top"].append(self.iid)
            
                # relationの情報を更新
                for data in self.relationInfoList.values():
                    for index, item in enumerate(data):
                        check1 = editValue + "->"
                        check2 = editValue + "."
                        temp = item
                        if check1 in item:
                            updateValue = newValue + "->"
                            # 古い値を削除
                            data.pop(index)
                            # 編集値を更新
                            update = temp.replace(check1, updateValue)
                            data.insert(index, update)
                        elif check2 in item:
                            updateValue = newValue + "."
                            # 古い値を削除
                            data.pop(index)
                            # 編集値を更新
                            update = temp.replace(check2, updateValue)
                            data.insert(index, update)

            # relation param 表示を更新する
            relationIndex = self.relationList.curselection()
            if len(relationIndex) != 0:
                selectedRelation = self.relationList.get(relationIndex)
                self.listboxRelationParams.set("")
                self.listboxRelationParams.set(self.relationInfoList[selectedRelation])

            messagebox.showinfo("確認", "更新しました。")

    def insertEntity(self):
        """ 子要素を追加処理

        """
        if len(self.addEntity.get()) == 0:
            messagebox.showerror("Error", "入力してください!")

        addItem = self.addEntity.get()

        # エンティティ名がない場合は処理しない
        if addItem != "":
            children = self.tree.get_children(self.iid)

            # 同じ階層に同じ名前で作成は不可
            for child in children:
                childname = self.tree.item(child, "text")
                if childname == addItem:
                    messagebox.showerror("登録エラー"," 既に登録されています")
                    return

            # fusekiに同期する
            # 唯一名を生成
            id = "".join(random.choice(
                string.ascii_letters + string.digits) for _ in range(20)) + "-1"

            # 選択しているentityのuriを取得する
            parentUri = self.getSelectedEntity()

            self.fusekiInsert.append([URIRef(self.entityUri+id), \
                        RDF.type, self.pd3.Object])
            self.fusekiInsert.append([URIRef(self.entityUri+id), \
                        self.pd3.partOf, URIRef(parentUri)])
            self.fusekiInsert.append([URIRef(self.entityUri+id), \
                        self.pd3.value, Literal(addItem)])
            self.fusekiInsert.append([URIRef(self.entityUri+id), \
                        self.pd3.type, Literal("TARGET")])

            if self.fuseki.insert(self.entityGraph, self.fusekiInsert) is False:
                messagebox.showerror("Error", "Fusekiにインサート失敗!")
            else:
                # treeに追加する
                insert_iid = self.tree.insert(self.iid, "end", text=addItem)
                parentText = self.tree.item(self.iid, "text")
                if self.iid not in self.entityTree.keys():
                    self.entityTree[self.iid] = []
                    self.treeUriList[parentText] = {}
                    self.treeIdTable[parentText] = {}

                self.entityTree[self.iid].append(insert_iid)
                self.treeUriList[parentText][addItem] = self.entityUri+id
                self.treeIdTable[parentText][addItem] = insert_iid
                # 追加したらEntryの値を削除する
                self.addEntity.set("")

            self.fusekiInsert.clear()

    def save(self):
        """ オントロジーデータを保存処理

        """
        dirList = []
        for line in self.tree.get_children():
            self.createDirectoryList(line, self.tree, dirList)

        self.fuseki.get_graph_ttlfile(self.entityGraph, self.savePathEntity.get())
        messagebox.showinfo("確認", "RDF ファイルを作成しました。")

    def createDirectoryList(self, rootiid, tree, dirList):
        """ ディレクトリリストを保存処理

        """
        dirname = tree.item(rootiid, "text")
        children = tree.get_children(rootiid)
        childlist = []
        dirList.append(dirname)
        if len(children) > 0:
            for child in children:
                self.createDirectoryList(child, tree, childlist)

    def selectProp(self, event):
        """ プロパティリストを選択イベント

        Args:
            event: イベント

        """
        w = event.widget
        select_prop_index = int(w.curselection()[0])
        select_prop_name = self.propNameList[select_prop_index]
        self.propName.set(select_prop_name)

        value = self.propInfo[self.iid][select_prop_name]
        self.propValue.set(value)

    def addProp(self):
        """ プロパティを追加処理

        """
        if len(self.propName.get()) == 0:
            messagebox.showerror("Error", "追加するプロパティを入力してください!")

        else:
            propName = self.propName.get()

            # Fusekiに同期する
            # uriを取得
            parentUri = self.getSelectedEntity()

            # 唯一idを生成する
            id = "".join(random.choice(
                string.ascii_letters + string.digits) for _ in range(20)) + "-1"
            subject = self.entityUri+id

            self.fusekiInsert.append([URIRef(subject), \
                        RDF.type, URIRef(self.pd3.parameter)])
            self.fusekiInsert.append([URIRef(subject), \
                        self.pd3.paramName, Literal(propName)])
            self.fusekiInsert.append([URIRef(subject), \
                        self.pd3.paramValue, Literal("None")]) 
            self.fusekiInsert.append([URIRef(subject), \
                        self.pd3.paramOf, URIRef(parentUri)]) 

            self.fusekiInsert.append([URIRef(parentUri), \
                        self.pd3.hasParameter, URIRef(subject)]) 

            if self.fuseki.insert(self.entityGraph, self.fusekiInsert) is False:
                messagebox.showerror("Error", "Fusekiにインサート失敗!")
            else:
                if self.iid not in self.propInfo.keys():
                    self.propInfo[self.iid] = {}
                    self.propInfoUri[self.iid] = {}

                self.propInfo[self.iid][propName] = None
                #同じプロパティ名があるため、uriを記録する
                self.propInfoUri[self.iid][propName] = subject

                self.propNameList.append(propName)

                self.propList.delete(0, tk.END)
                for item in self.propInfo[self.iid]:
                    self.propList.insert(tk.END, item)
                # propName入力内容を削除する
                self.propName.set("")

            self.fusekiInsert.clear()

    def editProp(self):
        """ プロパティを編集処理

        """
        selected_prop_index = self.propList.curselection()

        if len(selected_prop_index) == 0:
            messagebox.showerror("Error", "No proproty selected!")
        elif len(self.addPropNameEntry.get()) == 0:
            messagebox.showerror("Error", "No new name inputed")
        else:
            new_prop = self.addPropNameEntry.get()
            self.select_prop_name = self.propList.get(selected_prop_index)
            uri = self.propInfoUri[self.iid][self.select_prop_name]
            relationIndex = self.relationList.curselection()
            selectedRelation = self.relationList.get(relationIndex)

            # parametersと関連するrelationshipを取得
            relation = self.GetRelationByParam(uri)

            # Fuseki同期する
            updatePropQuery = """
                PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                WITH """ + self.entityGraph + """
                DELETE { ?s pd3:paramName """ + '"' + self.select_prop_name + '"' + """ }
                INSERT { ?s pd3:paramName """ + '"' + new_prop + '"' + """ }
                WHERE { ?s ?p ?o.
                        FILTER(?s = """ + '<' + uri + '>' + """)}""" 
 
            if self.fuseki.update(updatePropQuery):
                self.propInfo[self.iid][new_prop] = \
                    self.propInfo[self.iid][self.select_prop_name]
                del self.propInfo[self.iid][self.select_prop_name]

                self.propInfoUri[self.iid][new_prop] = uri
                del self.propInfoUri[self.iid][self.select_prop_name]

                self.propList.delete(selected_prop_index)
                self.propList.insert(selected_prop_index, self.addPropNameEntry.get())
                self.propNameList[selected_prop_index[0]] = self.addPropNameEntry.get()

                # relation param表示リスト更新
                for delData in relation:
                    for relationName, relationUri in self.relationUri.items():
                        if relationUri == delData:
                            for index, param in enumerate(self.relationInfoListUri[relationName]):
                                if param == uri:
                                    # Entity情報を取得
                                    select_entity = self.tree.item(self.iid, "text")
                                    parent_iid = self.tree.parent(self.iid)

                                    # root entityではない場合
                                    if len(parent_iid) != 0:
                                        parent_entity = self.tree.item(parent_iid, "text")
                                        addData = parent_entity + "->" + select_entity + "." + new_prop
                                    # root entityの場合
                                    else:
                                        addData = select_entity + "." + new_prop

                                    del self.relationInfoList[relationName][index]
                                    self.relationInfoList[relationName].insert(index, addData)

                            # relation param 表示を更新する
                            self.listboxRelationParams.set("")
                            self.listboxRelationParams.set(self.relationInfoList[selectedRelation])

                self.propName.set("")
                self.propValue.set("")
                messagebox.showinfo("確認", "更新しました。")

    def deleteProp(self):
        """ プロパティを削除処理

        """
        selected_prop_index = self.propList.curselection()

        if(len(selected_prop_index) == 0):
            messagebox.showerror("Error", "No proproty selected!")
        else:
            self.select_prop_name = self.propList.get(selected_prop_index)
            uri = self.propInfoUri[self.iid][self.select_prop_name]
            objectName = self.tree.item(self.iid, "text")

            # Fuseki同期する
            # parametersと関連するrelationshipを取得
            relation = self.GetRelationByParam(uri)

            # パラメータを削除
            deletePropQuery = """
               PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                WITH """ + self.entityGraph + """
                DELETE { ?s ?p ?o }
                WHERE { ?s ?p ?o.
                        FILTER(?s = """ + '<' + uri + '>' + """)}""" 

            # hasParameter情報を削除
            deleteHasParamQuery = """
               PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                WITH """ + self.entityGraph + """
                DELETE { ?s pd3:hasParameter """ + '<' + uri + '>' + """ }
                WHERE { ?s ?p ?o;
                        pd3:value ?value.
                        FILTER(?value = """ + '"' + objectName + '"' + """)}""" 

            for delData in relation:
                delRelationQuery = """
                    PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                    WITH """ + self.entityGraph + """
                    DELETE { ?s pd3:include """ + '<' + uri + '>' + """ }
                    WHERE { ?s ?p ?o.
                            FILTER(?s = """ + '<' + delData + '>' + """)}""" 

                if self.fuseki.update(delRelationQuery) is False:
                    messagebox.showerror("Error", "Fuseki削除失敗!")
                # relation param list 削除
                else: 
                    for relationName, relationUri in self.relationUri.items():
                        if relationUri == delData:
                            for index, param in enumerate(self.relationInfoListUri[relationName]):
                                if param == uri:
                                    del self.relationInfoList[relationName][index]                    
                                    del self.relationInfoListUri[relationName][index]

                            # relation param 表示を更新する
                            self.listboxRelationParams.set("")
                            self.listboxRelationParams.set(self.relationInfoList[relationName])

            if self.fuseki.update(deletePropQuery) and self.fuseki.update(deleteHasParamQuery) :
                self.propList.delete(selected_prop_index)
                del self.propInfo[self.iid][self.select_prop_name]
                del self.propNameList[selected_prop_index[0]]

                self.propName.set("")
                self.propValue.set("")

                messagebox.showinfo("確認", "削除しました。")

    def editPropvalue(self):
        """ プロパティバリューを編集処理

        """
        selected_prop_index = self.propList.curselection()
        if(len(selected_prop_index) == 0):
            messagebox.showerror("Error", "No proproty selected!")
        else:
            self.select_prop_name = self.propList.get(selected_prop_index)

            uri = self.propInfoUri[self.iid][self.select_prop_name]
            newValue = self.propValue.get()

            getValueQuery = """
                PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                SELECT DISTINCT ?value
                WHERE {
                    graph """ + self.entityGraph + """ {
                    ?node ?p ?o;
                        pd3:paramValue ?value.
                    FILTER(?node = """ + '<' + uri + '>' + """)
                    }
                }
            """
            query_results = self.fuseki.get_fuseki_data_json(getValueQuery)
            if len(query_results) > 0:
                for result in query_results:
                    oldValue = result["value"]["value"]

                updateValueQuery = """
                    PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                    WITH """ + self.entityGraph + """
                    DELETE { ?s pd3:paramValue """ + '"' + oldValue + '"' + """ }
                    INSERT { ?s pd3:paramValue """ + '"' + newValue + '"' + """ }
                    WHERE { ?s ?p ?o.
                            FILTER(?s = """ + '<' + uri + '>' + """)}""" 
                
                if self.fuseki.update(updateValueQuery) is True:
                    self.propInfo[self.iid][self.select_prop_name] = newValue

                    messagebox.showinfo("確認", "更新しました。")
            else:
                messagebox.showinfo("確認", "Parammeter value does not exit.")

    def initRelation(self):
        """ 初期化時、relation情報を取得

        """
        query= """
            PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
            SELECT ?relation ?name
            WHERE {
                graph """ + self.entityGraph + """ {
                ?relation ?p pd3:relation;
                    pd3:relationName ?name.
                }
            }
        """
        relationResults = self.fuseki.get_fuseki_data_json(query)
        if(len(relationResults) > 0):
            for result in relationResults:
                subject = result["relation"]["value"]
                name = result["name"]["value"]
                self.relationUri[name] = subject
                if name not in self.relationInfoList.keys():
                    self.relationInfoList[name] = []
                    self.relationInfoListUri[name] = []

        #relation paramsがあれば取得
        relationQuery = """
            PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
            SELECT ?name ?param
            WHERE {
                graph """ + self.entityGraph + """ {
                ?relation ?p pd3:relation;
                    pd3:relationName ?name;
                    pd3:include ?param.
                }
            }
        """
        query_result = self.fuseki.get_fuseki_data_json(relationQuery)
        if(len(query_result) > 0):
            for result in query_result:
                relationName = result["name"]["value"]
                param = result["param"]["value"]
                for entityId in self.propInfoUri.keys():
                    for key in self.propInfoUri[entityId]:
                        if(self.propInfoUri[entityId][key] == param):
                            paramName = key
                            # Entity情報を取得
                            select_entity = self.tree.item(entityId, "text")
                            parent_iid = self.tree.parent(entityId)

                            # root entityではない場合
                            if len(parent_iid) != 0:
                                parent_entity = self.tree.item(parent_iid, "text")
                                data = parent_entity + "->" + select_entity + "." + paramName
                            else:
                                data = select_entity + "." + paramName

                            paramSubject = self.propInfoUri[entityId][paramName]
                self.relationInfoList[relationName].append(data)
                self.relationInfoListUri[relationName].append(paramSubject)

        #リストに表示する
        for relation in self.relationInfoList.keys():
            self.relationList.insert(tk.END, relation)

    def selectRelation(self, event):
        """ relationリストを選択取得

        Args:
            event: イベント

        """
        selected_relation_index = self.relationList.curselection()
        relationData = self.relationList.get(selected_relation_index)
        self.relationName.set(relationData)

        self.relationParamsList.delete(0, tk.END)
        for item in self.relationInfoList[relationData]:
            self.relationParamsList.insert(tk.END, item)

    def addRelation(self):
        """ relationを追加処理

        """
        insertData = self.relationName.get()

        if len(insertData) == 0:
            messagebox.showerror("Error", "追加する関係名を入力してください!")
        else:
            if insertData in self.relationInfoList.keys():
                messagebox.showerror("Error", "既に登録されています")
            else:
                # Fusekiにインサート
                id = "".join(random.choice(
                    string.ascii_letters + string.digits) for _ in range(20)) + "-1"
                subject = self.entityUri+id

                self.fusekiInsert.append([URIRef(subject), \
                        RDF.type, URIRef(self.pd3.relation)])
                self.fusekiInsert.append([URIRef(subject), \
                        self.pd3.relationName, Literal(insertData)])

                if self.fuseki.insert(self.entityGraph, self.fusekiInsert) is False:
                    messagebox.showerror("Error", "Fusekiにインサート失敗!")
                else:
                    # 選択項目を解除する
                    self.relationList.select_clear(0, tk.END)

                    self.relationList.insert(tk.END, insertData)
                    self.relationInfoList[insertData] = []
                    # パラメータuriを記録のため
                    self.relationInfoListUri[insertData] = []
                    # 追加したら入力entryをクリア
                    self.relationName.set("")
                    self.listboxRelationParams.set("")

                    # uriを記録する
                    self.relationUri[insertData] = subject

                self.relationName.set("")
                self.fusekiInsert.clear()

    def editRelation(self):
        """ relationを編集処理

        """
        selected_relation_index = self.relationList.curselection()

        if len(selected_relation_index) == 0:
            messagebox.showerror("Error", "No relationship selected!")
        elif len(self.relationName.get()) == 0:
            messagebox.showerror("Error", "No new value inputed")
        else:
            new_value = self.relationName.get()
            old_value = self.relationList.get(selected_relation_index)

            uri = self.relationUri[old_value]

            # Fusekiとの同期
            updateQuery = """
                PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                WITH """ + self.entityGraph + """
                DELETE { ?s pd3:relationName """ + '"' + old_value + '"' + """ }
                INSERT { ?s pd3:relationName """ + '"' + new_value + '"' + """ }
                WHERE { ?s ?p ?o.
                        FILTER(?s = """ + '<' + uri + '>' + """)}""" 

            if self.fuseki.update(updateQuery) is False:
                    messagebox.showerror("Error", "Fuseki更新失敗!")
            else:
                # リストボックスを更新
                self.relationList.delete(selected_relation_index)
                self.relationList.insert(selected_relation_index, new_value)

                self.relationInfoList[new_value] = []
                for param in self.relationInfoList[old_value]:
                    self.relationInfoList[new_value].append(param)
                
                self.relationInfoListUri[new_value] = []
                for param in self.relationInfoListUri[old_value]:
                    self.relationInfoListUri[new_value].append(param)

                # relation uri を更新
                self.relationUri[new_value] = uri

                del self.relationInfoList[old_value]
                del self.relationInfoListUri[old_value]
                del self.relationUri[old_value]

                self.relationName.set("")
                self.listboxRelationParams.set("")

    def delRelation(self):
        """ relationを削除処理

        """
        selected_relation_index = self.relationList.curselection()

        if len(selected_relation_index) == 0:
            messagebox.showerror("Error", "No relationship selected!")
        else:
            delData = self.relationList.get(selected_relation_index)

            # uriを取得
            uri = self.relationUri[delData]

            # Fusekiとの同期
            deleteRelationQuery = """
                PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                WITH """ + self.entityGraph + """
                DELETE { ?s ?p ?o }
                WHERE { ?s ?p ?o.
                        FILTER(?s = """ + '<' + uri + '>' + """)}
            """

            if self.fuseki.update(deleteRelationQuery) is False:
                 messagebox.showerror("Error", "削除失敗!")
            else:
                for param in self.relationInfoListUri[delData]:
                    query = """
                        PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                        DELETE DATA
                        { GRAPH """ + self.entityGraph + """
                        { """ + "<" + param + ">" + """ pd3:satisfy """ + "<" + uri  + ">" + """  } } 
                    """

                    if self.fuseki.update(query) is False:
                        messagebox.showerror("Error", "削除失敗!")

                del self.relationInfoList[delData]
                del self.relationInfoListUri[delData]
                # listboxを削除
                self.relationList.delete(selected_relation_index)

                self.relationName.set("")
                # パラメータリストを削除
                self.relationParamsList.delete(0, tk.END)
                messagebox.showinfo("確認", "削除しました。")

    def addRelationParams(self):
        """ relation パラメータを追加処理

        """
        # プロパティ情報を取得
        selected_prop_index = self.propList.curselection()

        if len(selected_prop_index) == 0:
            propData = None
            messagebox.showerror("Error", "No proproty selected!")
        else:
            propData =  self.propList.get(selected_prop_index)
            # Entity情報を取得
            select_entity = self.tree.item(self.iid, "text")
            parent_iid = self.tree.parent(self.iid)

            # root entityではない場合
            if len(parent_iid) != 0:
                parent_entity = self.tree.item(parent_iid, "text")
                addData = parent_entity + "->" + select_entity + "." + propData
            else:
                addData = select_entity + "." + propData

        selected_relation_index = self.relationList.curselection()

        # relationship名を取得
        if len(selected_relation_index) == 0:
            relationData = None
            messagebox.showerror("Error", "No relationship selected!")
        else:
            relationData = self.relationList.get(selected_relation_index)

        if propData is not None and relationData is not None:
            #まだ登録されてない場合
            if addData not in self.relationInfoList[relationData]:

                # プロパティ主語を取得
                relationSubject = self.relationUri[relationData]
                propSubject = self.propInfoUri[self.iid][propData]

                # Fusekiにインサート
                self.fusekiInsert.append([URIRef(relationSubject), \
                            self.pd3.include, URIRef(propSubject)])
                self.fusekiInsert.append([URIRef(propSubject), \
                            self.pd3.satisfy, URIRef(relationSubject)])

                if self.fuseki.insert(self.entityGraph, self.fusekiInsert) is False:
                    messagebox.showerror("Error", "Fusekiにインサート失敗!")
                else:
                    self.listboxRelationParams.set("")
                    self.relationInfoList[relationData].append(addData)
                    self.relationInfoListUri[relationData].append(propSubject)
                    self.listboxRelationParams.set(self.relationInfoList[relationData])

                    self.fusekiInsert.clear()
            else:
                messagebox.showerror("Error", "既に登録されています!")

    def delRelationParams(self):
        """ relation パラメータを削除処理

        """

        #relationを取得
        selected_relation_index = self.relationList.curselection()
        relationData = self.relationList.get(selected_relation_index)

        #paramsを取得
        paramIndex = self.relationParamsList.curselection()

        if len(paramIndex) == 0:
            messagebox.showerror("Error", "No parameter selected!")
        else:
            paramData = self.relationParamsList.get(paramIndex)
            
            #uriを取得
            for index, data in enumerate(self.relationInfoList[relationData]):
                if data == paramData:
                    paramUri = self.relationInfoListUri[relationData][index]

            relationUri = self.relationUri[relationData]
            #fusekiとの同期
            delParamsQuery =  """
                PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                DELETE DATA
                { GRAPH """ + self.entityGraph + """
                { """ + "<" + relationUri + ">" + """ pd3:include """ + "<" + paramUri + ">" + """  } } 
            """

            delRelationQuery = """
                PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                DELETE DATA
                { GRAPH """ + self.entityGraph + """
                { """ + "<" + paramUri + ">" + """ pd3:satisfy """ + "<" + relationUri + ">" + """  } } 
            """

            if self.fuseki.update(delParamsQuery) and self.fuseki.update(delRelationQuery):
                self.relationInfoList[relationData].remove(paramData)
                self.relationParamsList.delete(paramIndex)

                messagebox.showinfo("確認", "削除しました。")
            else:
                messagebox.showerror("Error", "削除失敗!")

    def GetRelationByParam(self, paramUri):
        """ パラメータと関連あるリレーションを取得

        Args:
            paramUri: パラメータUri情報

        """
        relation = []
        query = """
            PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
            SELECT ?relation
            WHERE {
                graph """ + self.entityGraph + """ {
                ?s ?p ?o;
                    pd3:satisfy ?relation.
                FILTER(?s = """ + '<' + paramUri + '>' + """)}
            }""" 


        results = self.fuseki.get_fuseki_data_json(query)
        if len(results) > 0:
            for result in results:
                data = result["relation"]["value"]
                if data not in relation:
                    relation.append(result["relation"]["value"])

        return relation

    def getSelectedEntityProp(self):
        """ 選択したプロパティ情報を取得

        """
        selected_prop_index = self.propList.curselection()

        if len(selected_prop_index) == 0:
            uri = None
        else:
            # property
            self.select_prop_name = self.propList.get(selected_prop_index)
            uri = self.propInfoUri[self.iid][self.select_prop_name]
        
        return uri 

    def getSelectedEntity(self):
        """ 選択したエンティティ情報を取得

        """
        name = self.tree.item(self.iid, "text")
        parent_iid = self.tree.parent(self.iid)

        # root entityではない場合
        if len(parent_iid) != 0:
            for item in self.treeIdTable.keys():
                if item != "top":
                    for entity in self.treeIdTable[item]:
                        if entity == name and \
                            self.treeIdTable[item][entity] == self.iid:
                            parentUri = self.treeUriList[item][entity]
        else:
            for item in self.treeIdTable["top"]:
                if item == name and self.treeIdTable["top"][item] == self.iid:
                    parentUri = self.treeUriList["top"][item]

        return parentUri

    def insertInput(self):
        """ actionInput関係付けを追加

        """
        (current_item, current_node_id) = self.get_selected_node_id()

        if(current_item is not None) or (current_node_id is not None):
            node = self.gpm_uri + current_node_id

            propUri = self.getSelectedEntityProp()
            parentUri = self.getSelectedEntity()

            if propUri != None and parentUri != None:
                # fusekiに追加する
                self.fusekiInsert.append([URIRef(node), \
                            self.pd3.actionInput, URIRef(parentUri)])
                self.fusekiInsert.append([URIRef(node), \
                            self.pd3.actionInput, URIRef(propUri)])

                if self.fuseki.insert(self.selected_graph_name, self.fusekiInsert) is False:
                    messagebox.showerror("Error", "Fusekiにインサート失敗!")
                else: 
                    # 情報表示を更新
                    (input, output) = self.data.get_inputOutput_info(
                        self.selected_graph_name, node, current_node_id)
                    self.node_info_clear(self.actionInput_info)
                    self.node_info_update(self.actionInput_info, ', '.join(input))

                    messagebox.showinfo("確認", "Fusekiに追加しました。")
            else:
                messagebox.showerror("Error", "No data selected!")

    def insertOutput(self):
        """ outputInput関係付けを追加

        """
        (current_item, current_node_id) = self.get_selected_node_id()

        if(current_item is not None) or (current_node_id is not None):
            node = self.gpm_uri + current_node_id

            propUri = self.getSelectedEntityProp()
            parentUri = self.getSelectedEntity()

            if propUri != None and parentUri != None:
                # fusekiに追加する
                self.fusekiInsert.append([URIRef(node), \
                            self.pd3.actionOutput, URIRef(parentUri)])
                self.fusekiInsert.append([URIRef(node), \
                            self.pd3.actionOutput, URIRef(propUri)])

                if self.fuseki.insert(self.selected_graph_name, self.fusekiInsert) is False:
                    messagebox.showerror("Error", "Fusekiにインサート失敗!")
                else: 
                    # 情報表示を更新
                    (input, output) = self.data.get_inputOutput_info(
                        self.selected_graph_name, node, current_node_id)
                    self.node_info_clear(self.actionOutput_info)
                    self.node_info_update(self.actionOutput_info, ', '.join(output))

                    messagebox.showinfo("確認", "Fusekiに追加しました。")
            else:
                messagebox.showerror("Error", "No data selected!")

    def delInput(self):
        """ inputInput関係付けを削除

        """
        (current_item, current_node_id) = self.get_selected_node_id()

        if(current_item is not None) or (current_node_id is not None):
            node = self.gpm_uri + current_node_id

            propUri = self.getSelectedEntityProp()
            parentUri = self.getSelectedEntity()

            if propUri != None and parentUri != None:
                #　fuseki上のデータを削除する
                delQuery1 =  """
                    PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                    DELETE DATA
                    { GRAPH """ + self.selected_graph_name+ """
                    { """ + "<" + node + ">" + """ pd3:actionInput """ + "<" + parentUri + ">" + """  } } 
                """

                delQuery2 = """
                    PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                    DELETE DATA
                    { GRAPH """ + self.selected_graph_name + """
                    { """ + "<" + node + ">" + """ pd3:actionInput """ + "<" + propUri + ">" + """  } } 
                """

                if self.fuseki.update(delQuery1) and self.fuseki.update(delQuery2):
                    # 情報表示を更新
                    (input, output) = self.data.get_inputOutput_info(
                        self.selected_graph_name, node, current_node_id)
                    self.node_info_clear(self.actionInput_info)
                    self.node_info_update(self.actionInput_info, ', '.join(input))

                    messagebox.showinfo("確認", "削除しました。")
                else:
                    messagebox.showerror("Error", "削除失敗!")
            else:
                messagebox.showerror("Error", "No data selected!")

    def delOutput(self):
        """ outputInput関係付けを削除

        """
        (current_item, current_node_id) = self.get_selected_node_id()

        if(current_item is not None) or (current_node_id is not None):
            node = self.gpm_uri + current_node_id

            propUri = self.getSelectedEntityProp()
            parentUri = self.getSelectedEntity()

            if propUri != None and parentUri != None:
                #　fuseki上のデータを削除する
                delQuery1 =  """
                    PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                    DELETE DATA
                    { GRAPH """ + self.selected_graph_name+ """
                    { """ + "<" + node + ">" + """ pd3:actionOutput """ + "<" + parentUri + ">" + """  } } 
                """

                delQuery2 = """
                    PREFIX pd3: <http://DigitalTriplet.net/2021/08/ontology#>
                    DELETE DATA
                    { GRAPH """ + self.selected_graph_name + """
                    { """ + "<" + node + ">" + """ pd3:actionOutput """ + "<" + propUri + ">" + """  } } 
                """

                if self.fuseki.update(delQuery1) and self.fuseki.update(delQuery2):
                    # 情報表示を更新
                    (input, output) = self.data.get_inputOutput_info(
                        self.selected_graph_name, node, current_node_id)
                    self.node_info_clear(self.actionOutput_info)
                    self.node_info_update(self.actionOutput_info, ', '.join(output))

                    messagebox.showinfo("確認", "削除しました。")
                else:
                    messagebox.showerror("Error", "削除失敗!")
            else:
                messagebox.showerror("Error", "No data selected!")

In [ ]:
class logWindow(tk.Toplevel):
    """ ログウインドウクラス

    """
    def __init__(self, log_graph, log_uri, log_node, master=None):
        """  ログウィンドウの初期設定を行う

        Args:
            log_graph: ロググラフ
            log_uri: ログuri
            log_node: ログウインドウで選択したノード
            master:

        """
        super().__init__(master=master)
        # 主ウインドウを渡す変数
        self.log_graph = log_graph
        self.log_uri = log_uri
        self.log_node = log_node

        self.title("Log Window")
        self.log_frame = scroll.ScrollableFrame(self, canvas_width=1120, \
                                                canvas_height=600, \
                                                frame_width=1260, frame_height=760)
        self.log_frame.pack()
        self.set_log_tab()

        self.ttl_file_log = "ttl_file_log.ttl"
        self.fuseki = fuseki.FusekiManagement(setting.FUSEKI_URL, \
                                            setting.FUSEKI_DB)
        self.data = data_mg.DataManagement()

        # OSによってウィジェットの幅を調整する
        if platform.system() == "Windows":
            self.entryWidth = 14
            self.listBoxWidth = 26
        else:
            self.entryWidth = 9
            self.listBoxWidth = 21

        self.create_widgets()

    def set_log_tab(self):
        """  タブ作成

        """
        book_graph = ttk.Notebook(self.log_frame.scrollable_frame, width=650)
        book_graph.place(x=0, y=0)
        self.frame_tab_graph = ttk.Frame(book_graph, width=650, \
                                        height=560, relief="groove")
        book_graph.add(self.frame_tab_graph, text="Graph")

        book_action = ttk.Notebook(self.log_frame.scrollable_frame, \
                                    width=470, height=560)
        book_action.place(x=650, y=0)
        self.frame_tab_action = ttk.Frame(book_action, width=470, \
                                        height=580, relief="groove")
        book_action.add(self.frame_tab_action, text='Action')

    def create_widgets(self):
        """ ウィジェットを作成

        """
        # ------------------------------------------------------------------
        # LabelFrame
        # ------------------------------------------------------------------
        self.create_frame_widget()
        # ------------------------------------------------------------------
        # listbox widget (Graph Info)
        # ------------------------------------------------------------------
        self.scrollbar_ep_info_x = ttk.Scrollbar(self.frame_ep_info_list, \
                                                orient=tk.HORIZONTAL)
        self.scrollbar_ep_info_x.pack(side="bottom", fill="x")
        self.scrollbar_ep_info_y = ttk.Scrollbar(self.frame_ep_info_list, \
                                                orient=tk.VERTICAL)
        self.scrollbar_ep_info_y.pack(side="right", fill="y")

        self.listbox_ep_value = tk.StringVar()
        # graphを取得し、リストボックスに設定
        (self.graph_url_list, self.graph_list) = \
                                self.fuseki.get_graph(setting.FUSEKI_URL + "/")
        self.listbox_ep_value.set(self.graph_list)
        self.listbox_ep_info = tk.Listbox(self.frame_ep_info_list, \
                                        height=11, width=23, \
                                        selectmode=tk.SINGLE, \
                                        listvariable=self.listbox_ep_value, \
                                        exportselection=False, \
                                        xscrollcommand=self.scrollbar_ep_info_x.set, \
                                        yscrollcommand=self.scrollbar_ep_info_y.set)
        self.listbox_ep_info.pack(fill="both")
        self.listbox_ep_info.bind("<<ListboxSelect>>", self.select_graph_log)

        self.scrollbar_ep_info_x.config(command=self.listbox_ep_info.xview)
        self.scrollbar_ep_info_y.config(command=self.listbox_ep_info.yview)
        # ------------------------------------------------------------------
        # treeview widget
        # ------------------------------------------------------------------
        self.treeview = ttk.Treeview(self.frame_hierarchy_list, height=11)
        # 階層列設定
        self.treeview.column("#0", width=160, minwidth=500)
        # 階層列(ツリーカラム)の見出し設定。階層列とする。
        self.treeview.heading("#0")
        self.treeview.bind("<<TreeviewSelect>>", self.log_tree_select)
        self.treeview.grid(row=1, column=1, sticky="nsew")

        tree_scrollbar_y = ttk.Scrollbar(self.frame_hierarchy_list, \
                                        orient="vertical", \
                                        command=self.treeview.yview)
        tree_scrollbar_y.grid(row=1, column=2, sticky="nsew")
        self.treeview.configure(yscrollcommand=tree_scrollbar_y.set)

        tree_scrollbar_x = ttk.Scrollbar(self.frame_hierarchy_list, \
                                        orient="horizontal", \
                                        command=self.treeview.xview)
        tree_scrollbar_x.grid(row=2, column=1, sticky="nsew")
        self.treeview.configure(xscrollcommand=tree_scrollbar_x.set)
        # ------------------------------------------------------------------
        # ノード情報ウィジェットを作成
        # ------------------------------------------------------------------
        self.create_node_info_widget()
        # ------------------------------------------------------------------
        # listbox widget (GPM uri Info)
        # ------------------------------------------------------------------
        self.scrollbar_gpm_uri_list_x = ttk.Scrollbar(self.frame_new_gpm_list, \
                                                    orient=tk.HORIZONTAL)
        self.scrollbar_gpm_uri_list_x.pack(side="bottom", fill="x")
        self.scrollbar_gpm_uri_list_y = ttk.Scrollbar(self.frame_new_gpm_list, \
                                                    orient=tk.VERTICAL)
        self.scrollbar_gpm_uri_list_y.pack(side="right", fill="y")

        self.listbox_gpm_uri_list = tk.Listbox(self.frame_new_gpm_list, \
                                            height=11, width=23, \
                                            selectmode=tk.SINGLE, \
                                            exportselection=False, \
                                            xscrollcommand=self.scrollbar_gpm_uri_list_x.set, \
                                            yscrollcommand=self.scrollbar_gpm_uri_list_y.set)
        self.listbox_gpm_uri_list.pack(fill="both")

        self.scrollbar_gpm_uri_list_x.config(command=self.listbox_gpm_uri_list.xview)
        self.scrollbar_gpm_uri_list_y.config(command=self.listbox_gpm_uri_list.yview)
        # ------------------------------------------------------------------
        # listbox widget (object list)
        # ------------------------------------------------------------------
        self.scrollbar_object_x = ttk.Scrollbar(self.frame_object_list, \
                                            orient=tk.HORIZONTAL)
        self.scrollbar_object_x.pack(side="bottom", fill="x")
        self.scrollbar_object_y = ttk.Scrollbar(self.frame_object_list, \
                                            orient=tk.VERTICAL)
        self.scrollbar_object_y.pack(side="right", fill="y")

        self.listbox_object = tk.Listbox(self.frame_object_list, height=7, \
                                        width=self.listBoxWidth, \
                                        selectmode=tk.SINGLE, exportselection=False, \
                                        xscrollcommand=self.scrollbar_object_x.set, \
                                        yscrollcommand=self.scrollbar_object_y.set)
        self.listbox_object.pack(fill="both")
        self.listbox_object.bind("<<ListboxSelect>>", self.select_object)

        self.scrollbar_object_x.config(command=self.listbox_object.xview)
        self.scrollbar_object_x.config(command=self.listbox_object.yview)
        # ------------------------------------------------------------------
        # Button widget
        # ------------------------------------------------------------------
        # 表示EPを設定
        self.button_gpm_set = ttk.Button(self.frame_button, \
                                        text=" 表示 ", width=8, \
                                        command=self.set_gpm)
        self.button_gpm_set.grid(row=1, column=0, padx=7)
        # GUIを更新
        self.button_widget_renew = ttk.Button(self.frame_button, \
                                            text=" 更新 ", width=8, \
                                            command=self.use_renew_widget)
        self.button_widget_renew.grid(row=1, column=1, padx=7)
        # Log use関連付けノードを選択
        self.button_log_node_select = ttk.Button(self.frame_use_relation, \
                                                text=" 選択 ", width=7, \
                                                command=self.log_node_select)
        self.button_log_node_select.grid(row=1, column=0, padx=7)
        # ------------------------------------------------------------------
        # canvas
        # ------------------------------------------------------------------
        self.graph_canvas = tk.Canvas(self.frame_tab_graph)
        self.graph_canvas.place(x=5, y=5, width=455, height=550)
        # canvas スクロールバー
        self.canvas_bar_y = ttk.Scrollbar(self.graph_canvas, orient=tk.VERTICAL)
        self.canvas_bar_x = ttk.Scrollbar(self.graph_canvas, orient=tk.HORIZONTAL)
        self.canvas_bar_y.pack(side=tk.RIGHT, fill=tk.Y)
        self.canvas_bar_x.pack(side=tk.BOTTOM, fill=tk.X)
        self.canvas_bar_y.config(command=self.graph_canvas.yview)
        self.canvas_bar_x.config(command=self.graph_canvas.xview)
        self.graph_canvas.config(yscrollcommand=self.canvas_bar_y.set, \
                                xscrollcommand=self.canvas_bar_x.set)
        # Canvasのスクロール範囲を設定
        self.graph_canvas.config(scrollregion=(5, 5, 1000, 1000))

        # Action output
        self.node_info_input = ttk.Label(self.frame_ontology, \
                                        text="Action\nInput: ")
        self.node_info_input.grid(row=0, column=0)
        self.actionInput_info = scrolledtext.ScrolledText(self.frame_ontology, \
                                                        width=18, height=1, \
                                                        wrap=tk.WORD)
        self.actionInput_info.grid(row=0, column=1)
        self.actionInput_info.configure(state="disabled")

        # Action output
        self.node_info_output = ttk.Label(self.frame_ontology, \
                                        text="Action \nOutput:")
        self.node_info_output.grid(row=0, column=2)
        self.actionOutput_info = scrolledtext.ScrolledText(self.frame_ontology, \
                                                width=25, \
                                                height=1, wrap=tk.WORD)
        self.actionOutput_info.grid(row=0, column=3)
        self.actionOutput_info.configure(state="disabled")

    def create_frame_widget(self):
        """ LabelFrameを作成

        """
        # graph info
        self.frame_ep_info_list = ttk.LabelFrame(self.frame_tab_graph, \
                                                text=" Graph Info", \
                                                height=110, \
                                                style="style.TFrame")
        self.frame_ep_info_list.place(x=465, y=5)
        # Top Level URI List
        self.frame_new_gpm_list = ttk.LabelFrame(self.frame_tab_graph, \
                                                text="Top Level EP List", \
                                                height=190, \
                                                style="style.TFrame")
        self.frame_new_gpm_list.place(x=465, y=225)
        # Button
        self.frame_button = ttk.LabelFrame(self.frame_tab_graph, \
                                        text="Button", height=70, \
                                        style="style.TFrame")
        self.frame_button.place(x=465, y=445)
        # use関連付け
        self.frame_use_relation = ttk.LabelFrame(self.frame_tab_graph, \
                                                text="USE関連付け", \
                                                height=70, \
                                                style="style.TFrame")
        self.frame_use_relation.place(x=465, y=495)
        # node list
        self.frame_hierarchy_list = ttk.LabelFrame(self.frame_tab_action, \
                                                text="Action List", \
                                                style="style.TFrame")
        self.frame_hierarchy_list.grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)
        # node info
        self.frame_node = ttk.LabelFrame(self.frame_tab_action, \
                                        text="Infromation", \
                                        style="style.TFrame")
        self.frame_node.grid(row=0, column=1, rowspan=2, pady=5, sticky=tk.N)
        # object表示frame
        self.frame_object_list = ttk.LabelFrame(self.frame_tab_action, \
                                            text="Object List", \
                                            width=160, \
                                            height=150, \
                                            style="style.TFrame")
        self.frame_object_list.grid(row=1, column=0, padx=5, sticky=tk.W)

        # オントロジーデータ
        self.frame_ontology = ttk.LabelFrame(self.frame_tab_action, \
                                            text="Action Input/Output", \
                                            style="style.TFrame")
        self.frame_ontology.grid(row=2, column=0, columnspan=2, pady=10)

    def create_node_info_widget(self):
        """ ノード情報ウィジェットを作成

        """
        # ------------------------------------------------------------------
        # ノード情報
        # ------------------------------------------------------------------
        # id
        self.node_info_id = ttk.Label(self.frame_node, \
                                    text="Id:                 ")
        self.node_info_id.grid(row=0, column=0)
        self.id_info = tk.Text(self.frame_node, width=25, height=1)
        self.id_info.grid(row=0, column=1)
        self.id_info.configure(state="disabled")

        # action
        self.node_info_actionType = ttk.Label(self.frame_node, \
                                            text="ActionType:")
        self.node_info_actionType.grid(row=1, column=0)
        self.action_info = tk.Text(self.frame_node, width=25, \
                                height=1, wrap=tk.WORD)
        self.action_info.grid(row=1, column=1)
        self.action_info.configure(state="disabled")

        # layer
        self.node_info_layer = ttk.Label(self.frame_node, \
                                        text="Layer:          ")
        self.node_info_layer.grid(row=2, column=0)
        self.layer_info = tk.Text(self.frame_node, width=25, height=1, \
                                wrap=tk.WORD)
        self.layer_info.grid(row=2, column=1)
        self.layer_info.configure(state="disabled")

        # value
        self.node_info_value = ttk.Label(self.frame_node, \
                                      text="Value:           ")
        self.node_info_value.grid(row=3, column=0)
        self.value_info = scrolledtext.ScrolledText(self.frame_node, width=23, \
                                                    height=2, wrap=tk.WORD)
        self.value_info.grid(row=3, column=1)
        self.value_info.configure(state="disabled")

        # use
        self.node_info_use = ttk.Label(self.frame_node, \
                                    text="Use:              ")
        self.node_info_use.grid(row=4, column=0)
        self.use_info = scrolledtext.ScrolledText(self.frame_node, width=23, \
                                                height=2, wrap=tk.WORD)
        self.use_info.grid(row=4, column=1)
        self.use_info.configure(state="disabled")

        # useBy
        self.node_info_useby = ttk.Label(self.frame_node, \
                                        text="UseBy:          ")
        self.node_info_useby.grid(row=5, column=0)
        self.useby_info = scrolledtext.ScrolledText(self.frame_node, width=23, \
                                                    height=2, wrap=tk.WORD)
        self.useby_info.grid(row=5, column=1)
        self.useby_info.configure(state="disabled")

        # ------------------------------------------------------------------
        # 補足情報
        # ------------------------------------------------------------------
        # Intention
        self.node_info_intention = ttk.Label(self.frame_node, \
                                            text="Intention:    ")
        self.node_info_intention.grid(row=6, column=0)
        self.intention_info = scrolledtext.ScrolledText(self.frame_node, \
                                                        width=23, height=2, \
                                                        wrap=tk.WORD)
        self.intention_info.grid(row=6, column=1)
        self.intention_info.configure(state="disabled")

        # Rationale
        self.node_info_rationale = tk.Label(self.frame_node, \
                                            text="Rationale:    ")
        self.node_info_rationale.grid(row=7, column=0)
        self.rationale_info = scrolledtext.ScrolledText(self.frame_node, \
                                                        width=23, height=2, \
                                                        wrap=tk.WORD)
        self.rationale_info.grid(row=7, column=1)
        self.rationale_info.configure(state="disabled")

        # Annotation
        self.node_info_annotation = tk.Label(self.frame_node, \
                                            text="Annotation: ")
        self.node_info_annotation.grid(row=8, column=0)
        self.annotation_info = scrolledtext.ScrolledText(self.frame_node, \
                                                        width=23, height=2, \
                                                        wrap=tk.WORD)
        self.annotation_info.grid(row=8, column=1)
        self.annotation_info.configure(state="disabled")

        # Tool
        self.node_info_tool = tk.Label(self.frame_node, \
                                    text="Tool:   \t      ")
        self.node_info_tool.grid(row=9, column=0)
        self.tool_info = scrolledtext.ScrolledText(self.frame_node, width=23, \
                                                height=2, wrap=tk.WORD)
        self.tool_info.grid(row=9, column=1)
        self.tool_info.configure(state="disabled")

    def select_graph_log(self, event):
        """ グラフを選択処理

        Args:
            event: リスト選択イベント

        """
        w = event.widget
        select_index = int(w.curselection()[0])
        self.selected_graph = self.graph_url_list[select_index]

        self.selected_graph_name = "<" + self.selected_graph + ">"

        # treeviewを更新のため、元のtreeviewを削除
        if(self.treeview.get_children):
            for item in self.treeview.get_children():
                self.treeview.delete(item)

        # gpm listの選択をクリア
        self.listbox_gpm_uri_list.delete(0, tk.END)

        # ノード情報をクリア
        self.clear_node_info()

        (self.node_id_list) = \
                self.data.get_top_node_list_in_graph(self.selected_graph_name)

        self.get_uri_list(self.node_id_list)

        # グラフを表示のため、ttlファイルを取得
        self.fuseki.get_graph_ttlfile(self.selected_graph_name, self.ttl_file_log)

    def clear_node_info(self):
        """ ノード情報表示をクリア処理

        """
        self.node_info_clear(self.id_info)
        self.node_info_clear(self.action_info)
        self.node_info_clear(self.layer_info)
        self.node_info_clear(self.value_info)
        self.node_info_clear(self.use_info)
        self.node_info_clear(self.useby_info)

        self.node_info_clear(self.intention_info)
        self.node_info_clear(self.rationale_info)
        self.node_info_clear(self.annotation_info)
        self.node_info_clear(self.tool_info)

        self.node_info_clear(self.actionInput_info)
        self.node_info_clear(self.actionOutput_info)

    def show_top_log_graph(self):
        """ グラフのTOP部分を表示

        """
        # グラフ情報を取得
        (self.graph_node_list, self.graph_edge_list, \
                self.graph_label_list, self.graph_position) = \
            self.data.get_graph_data_by_ttlfile(self.gpm_uri, self.node_id_list["top"], \
                self.object_id_list["top"], None, self.ttl_file_log)

        # 最上層グラフを表示
        image_path = "top_log_graph.png"
        self.graph_level = "top"
        set_image(image_path, self.graph_node_list, self.graph_edge_list, \
                self.graph_label_list, [], \
                self.graph_position)
        top_image = Image.open(image_path)

        self.graph_canvas.photo = ImageTk.PhotoImage(top_image)
        self.image_on_canvas = self.graph_canvas.create_image(
            -100, -100, anchor='nw', image=self.graph_canvas.photo)

    def get_hierarchy_node_list(self):
        """ ツリーリスト階層ノード情報設定

        """
        # グラフにあるコンテナフローを取得
        container_count = self.data.get_graph_containerflow_count(
            self.selected_graph_name, self.gpm_uri)

        (self.node_id_list) = self.data.get_top_node_list_in_uri(
            self.selected_graph_name, self.gpm_uri)

        (self.object_list, self.object_id_list) = self.data.get_top_level_object_list(
            self.selected_graph_name, self.gpm_uri)

        self.node_list = {}
        self.node_list["top"] = []
        self.treeview_item_list = {}
        self.treeview_item_list["top"] = []

        for node_id in self.node_id_list["top"]:
            node_value = self.data.get_node_value(self.selected_graph_name, \
                self.gpm_uri, node_id)
            self.node_list["top"].append(node_value)

        for index, node in enumerate(self.node_list["top"]):
            parent = self.treeview.insert("", tk.END, text=node, open=False)
            self.treeview_item_list["top"].append(parent)
            self.get_member_node_list(
                self.gpm_uri, self.node_id_list["top"][index], container_count, parent)

    def get_member_node_list(self, uri, top_node, count, parent_node):
        """ ツリーリストmember情報を取得

        Args:
            uri: 選択したGPMのuri
            top_node: member情報を取得したいトップノードのid
            count: GPMにあるコンテナ数
            parent_node: ツリーリストの親ノード

        Returns:

        """
        if(count == 0):
            return

        top_node = top_node.replace("<br>", "")

        query_result = self.data.get_container_query(self.selected_graph_name, \
                                                uri + top_node)

        if(len(query_result) > 0):
            self.node_list[top_node] = []
            self.node_id_list[top_node] = []
            self.object_list[top_node] = []
            self.object_id_list[top_node] = []
            self.treeview_item_list[top_node] = []
            count -= 1
            for result in query_result:
                container = result["source"]["value"]

            # Containerのメンバーを取得
            query_result = self.data.get_container_member_query(
                            self.selected_graph_name, container, uri)

            for result in query_result:
                value = result["value"]["value"]
                id = result["id"]["value"]
                self.node_list[top_node].append(value)
                self.node_id_list[top_node].append(id)
                # return self.get_member_node_list(value, count)

            # Containerにあるオブジェクトを取得
            query_object_result = self.data.get_obejct_in_container_query(
                                    self.selected_graph_name, container, uri)

            for result in query_object_result:
                id = result["id"]["value"]
                value = result["value"]["value"]
                self.object_list[top_node].append(value)
                self.object_id_list[top_node].append(id)

            for index, node in enumerate(self.node_list[top_node]):
                child = self.treeview.insert(
                    parent_node, tk.END, text=node, open=False)
                self.treeview_item_list[top_node].append(child)
                node_uri = self.data.get_node_uri(
                    self.selected_graph_name, self.node_id_list[top_node][index])
                self.get_member_node_list(
                    node_uri, self.node_id_list[top_node][index], count, child)
        else:
            return

    def get_uri_list(self, node_id_list):
        """ top levelノードのuri listを取得

        Args:
            node_id_list: トップノードのidリスト

        Returns:

        """
        self.top_node_list_uri = {}
        self.con_ep_list = []
        for node_id in node_id_list["top"]:
            index = 0

            # nodeのuriを取得
            query_result = self.data.get_node_subject_id_query(
                            self.selected_graph_name, node_id)

            for result in query_result:
                temp_subject = result["s"]["value"]
                id = result["id"]["value"]
                node_subject = temp_subject.replace(id, "")
                if(node_subject not in self.top_node_list_uri):
                    self.top_node_list_uri[node_subject] = []
                    self.listbox_gpm_uri_list.insert(index, node_subject)
                    # container ep listに追加
                    self.con_ep_list.append(node_subject)
                    index += 1
                self.top_node_list_uri[node_subject].append(node_id)

    def log_tree_select(self, event):
        """ treeviewを選択処理

        Args:
            event: ツリーリスト選択イベント

        """
        # node名を取得
        record_id = self.treeview.focus()
        current_item = self.treeview.item(record_id)["text"]

        # node idを取得
        for key in self.treeview_item_list.keys():
            for index, item in enumerate(self.treeview_item_list[key]):
                if(item == record_id):
                    node_id = self.node_id_list[key][index]
                    new_graph_level = key

        # ノード情報表示をクリア
        self.clear_node_info()

        # オブジェクト表示リストをクリア
        self.listbox_object.delete(0, tk.END)

        # ノード情報を取得(id, actionType, layer, position など)
        (id, action, layer) = \
                    self.data.get_node_info(self.selected_graph_name, node_id)

        (use, useby) = \
            self.data.get_use_info(self.selected_graph_name, self.gpm_uri, node_id)

        # ActionInput, ActionOutputを取得
        (actionInput, actionOutput) = \
            self.data.get_inputOutput_info(self.selected_graph_name, self.gpm_uri, node_id)

        # ノード情報を表示
        self.node_info_update(self.id_info, id)
        self.node_info_update(self.action_info, action)
        self.node_info_update(self.layer_info, layer)
        self.node_info_update(self.value_info, current_item)

        self.node_info_update(self.use_info, ", ".join(use))
        self.node_info_update(self.useby_info, ", ".join(useby))
        self.node_info_update(self.actionInput_info, ", ".join(actionInput))
        self.node_info_update(self.actionOutput_info, ", ".join(actionOutput))

        # クエリを投げるため、ノードの主語を取得
        target_node = self.data.get_node_subject_info(self.selected_graph_name, node_id)
        # 補足情報を取得と表示(intention, annotation, rationale, objectなど)
        self.set_supplement_info(target_node)

        image_path = "selected_log_graph.png"
        # 選択したノードの層と表示しているグラフの層が異なる場合、グラフ更新
        if(new_graph_level != self.graph_level):
            self.graph_level = new_graph_level

            (self.graph_node_list, self.graph_edge_list, \
                self.graph_label_list, self.graph_position) = \
                self.data.get_graph_data_by_ttlfile(self.gpm_uri, \
                    self.node_id_list[self.graph_level], \
                    self.object_id_list[self.graph_level], None, self.ttl_file_log)

        # 画像を設定
        set_image(image_path, self.graph_node_list, self.graph_edge_list, \
                self.graph_label_list, [node_id], \
                self.graph_position)

        # 画像を更新
        selected_image = Image.open(image_path)
        self.change_image(selected_image)

    def set_supplement_info(self, node):
        """ ノード補足情報を取得と表示処理

        Args:
            node: 選択したノード(uri + node id)

        """
        # ------------------------------------------------------------------
        # intentionを取得
        # ------------------------------------------------------------------
        (intention_id, intention_value) = self.data.get_supplement_detail_info(
            self.selected_graph_name, "intention", node)

        if(intention_id is not None) and (intention_value is not None):
            self.node_info_update(self.intention_info, intention_value)

        # ------------------------------------------------------------------
        # rationaleを取得
        # ------------------------------------------------------------------
        (rationale_id, rationale_value) = self.data.get_supplement_detail_info(
            self.selected_graph_name, "rationale", node)

        if(rationale_id is not None) and (rationale_value is not None):
            self.node_info_update(self.rationale_info, rationale_value)

        # ------------------------------------------------------------------
        # annotationを取得
        # ------------------------------------------------------------------
        (annotation_id, annotation_value) = self.data.get_supplement_detail_info(
            self.selected_graph_name, "annotation", node)

        if(annotation_id is not None) and (annotation_value is not None):
            self.node_info_update(self.annotation_info, annotation_value)

        # ------------------------------------------------------------------
        # toolを取得
        # ------------------------------------------------------------------
        (tool_id, tool_value) = self.data.get_supplement_detail_info(
            self.selected_graph_name, "tool/knowledge", node)

        if(tool_value is not None):
            self.node_info_update(self.tool_info, tool_value)

        # ------------------------------------------------------------------
        # objectを取得
        # ------------------------------------------------------------------
        (object_id, object_value) = self.data.get_supplement_detail_info(
            self.selected_graph_name, "", node)

        if(object_id is not None) and (object_value is not None):
            for index, item in enumerate(object_value[node]):
                self.listbox_object.insert(index, item)

    def change_image(self, img):
        """ グラフ画像の入り替え処理

        Args:
            img: 更新する画像

        """
        # canvasの書き換え
        self.graph_canvas.photo = ImageTk.PhotoImage(img)
        self.graph_canvas.itemconfig(
            self.image_on_canvas, image=self.graph_canvas.photo)

    def log_node_select(self):
        """ ログノード選択処理

        """
        # node名を取得
        record_id = self.treeview.focus()
        if(len(record_id) == 0):
            messagebox.showerror("Error", "No data selected!", parent=self)
        else:
            current_item = self.treeview.item(record_id)["text"]

            # node idを取得
            for key in self.treeview_item_list.keys():
                for index, item in enumerate(self.treeview_item_list[key]):
                    if(item == record_id):
                        node_id = self.node_id_list[key][index]

            # 選択したノードのuriを取得
            node_uri = self.data.get_node_uri(self.selected_graph_name, node_id)

            # 主ウインドウに設定する
            self.log_graph.set(self.selected_graph_name)
            self.log_uri.set(node_uri)
            self.log_node.set(node_id)

    def set_gpm(self):
        """ treeviewとグラフで確認したいGPMを設定

        """
        selected_GPM_index = self.listbox_gpm_uri_list.curselection()
        if(len(selected_GPM_index) == 0):
            messagebox.showerror("Error", "No data selected!", parent=self)
        else:
            self.gpm_uri = self.listbox_gpm_uri_list.get(selected_GPM_index)

            # treeviewを更新のため、元のtreeviewを削除
            if(self.treeview.get_children):
                for item in self.treeview.get_children():
                    self.treeview.delete(item)

            # ノード情報をクリア
            self.clear_node_info()

            # オブジェクト表示リストをクリア
            self.listbox_object.delete(0, tk.END)

            # treeviewを表示
            self.get_hierarchy_node_list()

            # グラフを表示
            self.show_top_log_graph()

    def insert_member_event(self):
        """ コンテナとして埋め込み処理

        """
        # treeviewで選択したノード名とidを取得
        (current_item, current_node_id) = self.get_selected_node_id()

        if(current_item is not None) or (current_node_id is not None):
            # treeviewで選択したノードのuriを取得
            node_uri = self.data.get_node_uri(self.selected_graph_name, current_node_id)

            # member uriを取得
            selected_member_index = self.listbox_con_ep.curselection()
            if(len(selected_member_index) == 0):
                messagebox.showerror(
                    "Error", "No uri data selected!", parent=self)
            else:
                member_uri = self.listbox_con_ep.get(selected_member_index)

                # ----------------------------------------------------
                # 新uriで関連付けする両プロセスをコピーし、
                # 新uriのデータでコンテナ関連付けする
                # ----------------------------------------------------
                # 新uriを生成する
                now = datetime.datetime.now()
                new_uri = "http://localhost/Container_add_ep" + \
                            now.strftime("%Y%m%d_%H%M%S") + "/"
                new_prefix = "GPM_Container_" + now.strftime("%Y%m%d_%H%M%S")

                assoc.copy_ep_data(self.selected_graph_name, node_uri, new_uri, self)
                assoc.copy_ep_data(self.selected_graph_name, member_uri, new_uri, self)

                assoc.insert_member_new(self.selected_graph_name, current_item, \
                                current_node_id, self.selected_graph_name, member_uri, \
                                new_uri, self)

                # ウィジェット更新
                self.update_after_member_modfiy()

    def delete_member_event(self):
        """ コンテナメンバー削除処理

        """
        # treeviewで選択したノードnode名とidを取得
        (current_item, current_node_id) = self.get_selected_node_id()

        if(current_item is not None) or (current_node_id is not None):
            # 選択したノードがコンテナを含まれてない場合、エラーメッセージ
            if not current_node_id in self.node_list.keys():
                messagebox.showerror(
                    "Error", "選択したアクションがメンバーはありません!", parent=self)
            # 選択したノードがコンテナを含まれている場合、メンバー削除処理
            else:
                # メンバーリストを取得
                member_id_list = self.node_id_list[current_node_id]
                member_list = self.node_list[current_node_id]
                # オブジェクトリストを取得
                object_id_list = self.object_id_list[current_node_id]

                # コピーしてから削除する
                now = datetime.datetime.now()
                new_uri = "http://localhost/Container_delete_" + \
                    now.strftime("%Y%m%d_%H%M%S") + "/"

                assoc.copy_ep_data(self.selected_graph_name, self.gpm_uri, new_uri, self)

                assoc.delete_member(self.selected_graph_name, self.gpm_uri, new_uri, \
                            current_item, current_node_id, self.node_list, \
                            self.node_id_list, object_id_list, self)
                # ウィジェット更新
                self.update_after_member_modfiy()

    def node_info_update(self, widget, data):
        """ ノード情報更新処理

        Args:
            widget: 更新するウェジット
            data: 更新するデータ

        """
        widget.configure(state="normal")
        widget.insert(1.0, data)
        widget.configure(state="disabled")

    def node_info_clear(self, widget):
        """ ノード情報クリア処理

        Args:
            widget: 更新するウェジット

        """
        widget.configure(state="normal")
        widget.delete("1.0", "end")
        widget.configure(state="disabled")

    def select_object(self, event):
        """ object id listを選択処理

        Args:
            event: リスト選択イベント

        """
        w = event.widget
        select_object_index = int(w.curselection()[0])
        object_value = self.listbox_object.get(select_object_index)

        for key in self.object_list.keys():
            for index, value in enumerate(self.object_list[key]):
                if(value == object_value):
                    object_id = self.object_id_list[key][index]

        # グラフ表示
        image_path = "selected_graph.png"

        (self.graph_node_list, self.graph_edge_list, \
            self.graph_label_list, self.graph_position) = \
            self.data.get_graph_data_by_ttlfile(self.gpm_uri, \
                                    self.node_id_list[self.graph_level], \
                                    self.object_id_list[self.graph_level], \
                                    None, self.ttl_file_log)

        # 画像を設定
        set_image(image_path, self.graph_node_list, self.graph_edge_list, \
                self.graph_label_list, [object_id], \
                self.graph_position)

        # 画像を更新
        selected_image = Image.open(image_path)
        self.change_image(selected_image)

    def get_selected_node_id(self):
        """ treeviewで選択したノードのidを取得

        Returns:
            current_item: ノード名
            current_node_id: ノードid

        """
        current_item = None
        current_node_id = None
        # ノードnode名を取得
        record_id = self.treeview.focus()
        if(not record_id):
            messagebox.showerror("Error", "No node selected!", parent=self)
        else:
            current_item = self.treeview.item(record_id)["text"]

            # ノードidを取得
            for key in self.treeview_item_list.keys():
                for index, item in enumerate(self.treeview_item_list[key]):
                    if(record_id == item):
                        current_node_id = self.node_id_list[key][index]

        return current_item, current_node_id

    def update_after_member_modfiy(self):
        """ メンバー修正後更新処理

        Top level ep listを更新, treeview, ノード情報, container ep list, 
        canvasをクリア

        """
        # treeviewを更新のため、元のtreeviewを削除
        if(self.treeview.get_children):
            for item in self.treeview.get_children():
                self.treeview.delete(item)

        # gpm listの選択をクリア
        self.listbox_gpm_uri_list.delete(0, tk.END)

        # ノード情報をクリア
        self.clear_node_info()

        # オブジェクト表示リストをクリア
        self.listbox_object.delete(0, tk.END)

        (self.node_id_list) = \
                self.data.get_top_node_list_in_graph(self.selected_graph_name)

        self.get_uri_list(self.node_id_list)

        # canvasクリア
        self.graph_canvas.delete("all")

    def use_renew_widget(self):
        """ use内容を更新処理

        """
        # treeviewで選択したノードnode名とidを取得
        (current_item, current_node_id) = self.get_selected_node_id()

        # ノード情報表示を更新
        (use, useby) = \
            self.data.get_use_info(self.selected_graph_name, \
                                    self.gpm_uri, current_node_id)
        self.node_info_clear(self.use_info)
        self.node_info_update(self.use_info, ", ".join(use))



In [ ]:
class window_2(tk.Toplevel):
    """ GPMウインドウ2クラス

    """
    def __init__(self, win2_graph, win2_uri, win2_selected_item, master=None):
        """  GPMウインドウ2の初期設定を行う

        Args:
            win2_graph: GPM2グラフ
            win2_uri: GPM2のuri
            win2_selected_item: GPMウインドウ2で選択したノード
            master:

        """
        super().__init__(master=master)
        # 主ウインドウを渡す変数
        self.win2_graph = win2_graph
        self.win2_uri = win2_uri
        self.win2_selected_item = win2_selected_item

        self.title("GPM Window 2")
        # メニューバー作成 
        self.win2_frame = scroll.ScrollableFrame(self, canvas_width=880, \
                                                canvas_height=600, \
                                                frame_width=930, frame_height=760)
        self.win2_frame.pack()
        
        # タブ設定
        self.set_win2_tab()

        self.ttl_file_win2 = "ttl_file_win_2.ttl"
        self.fuseki = fuseki.FusekiManagement(setting.FUSEKI_URL, \
                                            setting.FUSEKI_DB)
        self.data = data_mg.DataManagement()

        self.create_widgets()

    def set_win2_tab(self):
        """  タブ作成

        """
        book_graph = ttk.Notebook(self.win2_frame.scrollable_frame, width=650)
        book_graph.place(x=0, y=0)
        self.frame_tab_graph = ttk.Frame(book_graph, width=650, \
                                        height=560, relief="groove")
        book_graph.add(self.frame_tab_graph, text="Graph")

        book_action = ttk.Notebook(self.win2_frame.scrollable_frame, width=220)
        book_action.place(x=650, y=-1)
        self.frame_tab_action = ttk.Frame(book_action, width=220, \
                                        height=560, relief="groove")
        book_action.add(self.frame_tab_action, text='Action')

    def create_widgets(self):
        """ ウィジェットを作成

        """
        # ------------------------------------------------------------------
        # LabelFrame
        # ------------------------------------------------------------------
        self.create_frame_widget()
        # ------------------------------------------------------------------
        # listbox widget (Graph Info)
        # ------------------------------------------------------------------
        self.scrollbar_ep_info_x = ttk.Scrollbar(self.frame_ep_info_list, \
                                                orient=tk.HORIZONTAL)
        self.scrollbar_ep_info_x.pack(side="bottom", fill="x")
        self.scrollbar_ep_info_y = ttk.Scrollbar(self.frame_ep_info_list, \
                                                orient=tk.VERTICAL)
        self.scrollbar_ep_info_y.pack(side="right", fill="y")

        self.listbox_ep_value = tk.StringVar()
        # graphを取得し、リストボックスに設定
        (self.graph_url_list, self.graph_list) = \
                                self.fuseki.get_graph(setting.FUSEKI_URL + "/")
        self.listbox_ep_value.set(self.graph_list)
        self.listbox_ep_info = tk.Listbox(self.frame_ep_info_list, height=12, \
                                        width=23, selectmode=tk.SINGLE, \
                                        listvariable=self.listbox_ep_value, \
                                        exportselection=False, \
                                        xscrollcommand=self.scrollbar_ep_info_x.set, \
                                        yscrollcommand=self.scrollbar_ep_info_y.set)
        self.listbox_ep_info.pack(fill="both")
        self.listbox_ep_info.bind("<<ListboxSelect>>", self.select_graph_win2)

        self.scrollbar_ep_info_x.config(command=self.listbox_ep_info.xview)
        self.scrollbar_ep_info_y.config(command=self.listbox_ep_info.yview)
        # ------------------------------------------------------------------
        # treeview widget
        # ------------------------------------------------------------------
        self.treeview = ttk.Treeview(self.frame_hierarchy_list)
        # 階層列設定
        self.treeview.column("#0", width=160, minwidth=500)
        # 階層列(ツリーカラム)の見出し設定。階層列とする。
        self.treeview.heading("#0")
        self.treeview.bind("<<TreeviewSelect>>", self.win2_tree_select)
        self.treeview.grid(row=1, column=1, sticky="nsew")

        tree_scrollbar_y = ttk.Scrollbar(self.frame_hierarchy_list, \
                                        orient="vertical", \
                                        command=self.treeview.yview)
        tree_scrollbar_y.grid(row=1, column=2, sticky="nsew")
        self.treeview.configure(yscrollcommand=tree_scrollbar_y.set)

        tree_scrollbar_x = ttk.Scrollbar(self.frame_hierarchy_list, \
                                        orient="horizontal", \
                                        command=self.treeview.xview)
        tree_scrollbar_x.grid(row=2, column=1, sticky="nsew")
        self.treeview.configure(xscrollcommand=tree_scrollbar_x.set)

        # ------------------------------------------------------------------
        # listbox widget (GPM uri Info)
        # ------------------------------------------------------------------
        self.scrollbar_gpm_uri_list_x = ttk.Scrollbar(self.frame_new_gpm_list, \
                                                    orient=tk.HORIZONTAL)
        self.scrollbar_gpm_uri_list_x.pack(side="bottom", fill="x")
        self.scrollbar_gpm_uri_list_y = ttk.Scrollbar(self.frame_new_gpm_list, \
                                                    orient=tk.VERTICAL)
        self.scrollbar_gpm_uri_list_y.pack(side="right", fill="y")

        self.listbox_gpm_uri_list = tk.Listbox(self.frame_new_gpm_list, \
                                            height=12, width=23, \
                                            selectmode=tk.SINGLE, \
                                            exportselection=False, \
                                            xscrollcommand=self.scrollbar_gpm_uri_list_x.set, \
                                            yscrollcommand=self.scrollbar_gpm_uri_list_y.set)
        self.listbox_gpm_uri_list.pack(fill="both")

        self.scrollbar_gpm_uri_list_x.config(command=self.listbox_gpm_uri_list.xview)
        self.scrollbar_gpm_uri_list_y.config(command=self.listbox_gpm_uri_list.yview)

        # ------------------------------------------------------------------
        # Button widget
        # ------------------------------------------------------------------
        # 表示EPを設定
        self.button_gpm_set = ttk.Button(self.frame_button, text=" 表示 ", \
                                        width=8, command=self.set_gpm)
        self.button_gpm_set.grid(row=1, column=0, padx=7)
        # GUIを更新
        self.button_widget_renew = ttk.Button(self.frame_button, text=" 更新 ", \
                                            width=8, \
                                            command=self.renew_widget)
        self.button_widget_renew.grid(row=1, column=1, padx=7)
        # メンバー関連付けEPを選択
        self.button_ep_select = ttk.Button(self.frame_member_relation, \
                                        text="EP全体を選択", width=12, \
                                        command=self.win2_ep_select)
        self.button_ep_select.grid(row=0, column=0, padx=5)
        # メンバー関連付けノードを選択
        self.button_node_select = ttk.Button(self.frame_member_relation, \
                                            text="コンテナを選択", width=12,
                                            command=self.win2_node_select)
        self.button_node_select.grid(row=0, column=1, padx=5)

        # ------------------------------------------------------------------
        # canvas
        # ------------------------------------------------------------------
        self.graph_canvas = tk.Canvas(self.frame_tab_graph)
        self.graph_canvas.place(x=5, y=5, width=455, height=550)
        # canvas スクロールバー
        self.canvas_bar_y = ttk.Scrollbar(self.graph_canvas, orient=tk.VERTICAL)
        self.canvas_bar_x = ttk.Scrollbar(self.graph_canvas, orient=tk.HORIZONTAL)
        self.canvas_bar_y.pack(side=tk.RIGHT, fill=tk.Y)
        self.canvas_bar_x.pack(side=tk.BOTTOM, fill=tk.X)
        self.canvas_bar_y.config(command=self.graph_canvas.yview)
        self.canvas_bar_x.config(command=self.graph_canvas.xview)
        self.graph_canvas.config(yscrollcommand=self.canvas_bar_y.set, \
                                xscrollcommand=self.canvas_bar_x.set)
        # Canvasのスクロール範囲を設定
        self.graph_canvas.config(scrollregion=(5, 5, 1000, 1000))

    def create_frame_widget(self):
        """ LabelFrameを作成

        """
        # graph info
        self.frame_ep_info_list = ttk.LabelFrame(self.frame_tab_graph, \
                                                text=" Graph Info", \
                                                height=120, \
                                                style="style.TFrame")
        self.frame_ep_info_list.place(x=465, y=5)
        # Top Level URI List
        self.frame_new_gpm_list = ttk.LabelFrame(self.frame_tab_graph, \
                                                text="Top Level EP List", \
                                                height=200, \
                                                style="style.TFrame")
        self.frame_new_gpm_list.place(x=465, y=240)
        # Button
        self.frame_button = ttk.LabelFrame(self.frame_tab_graph, \
                                        text="Button", height=70, \
                                        style="style.TFrame")
        self.frame_button.place(x=465, y=480)
        # node list
        self.frame_hierarchy_list = ttk.LabelFrame(self.frame_tab_action, \
                                                text="Action List", \
                                                height=240, \
                                                style="style.TFrame")
        self.frame_hierarchy_list.grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)

        # メンバー関連付け選択
        self.frame_member_relation = ttk.LabelFrame(self.frame_tab_action, \
                                                text="メンバー関連付け", \
                                                height=70, \
                                                style="style.TFrame")
        self.frame_member_relation.grid(row=1, column=0, padx=5, pady=15, sticky=tk.W)

    def select_graph_win2(self, event):
        """ グラフを選択処理

        Args:
            event: リスト選択イベント

        """
        w = event.widget
        select_index = int(w.curselection()[0])
        self.selected_graph = self.graph_url_list[select_index]

        self.selected_graph_name = "<" + self.selected_graph + ">"

        # treeviewを更新のため、元のtreeviewを削除
        if(self.treeview.get_children):
            for item in self.treeview.get_children():
                self.treeview.delete(item)

        # gpm listの選択をクリア
        self.listbox_gpm_uri_list.delete(0, tk.END)

        (self.node_id_list) = \
                self.data.get_top_node_list_in_graph(self.selected_graph_name)

        self.get_uri_list(self.node_id_list)

        # グラフを表示のため、ttlファイルを取得
        self.fuseki.get_graph_ttlfile(self.selected_graph_name, self.ttl_file_win2)

    def get_uri_list(self, node_id_list):
        """ top levelノードのuri listを取得

        Args:
            node_id_list: トップノードのidリスト

        Returns:

        """
        self.top_node_list_uri = {}
        self.con_ep_list = []
        for node_id in node_id_list["top"]:
            index = 0

            # nodeのuriを取得
            query_result = self.data.get_node_subject_id_query(
                            self.selected_graph_name, node_id)

            for result in query_result:
                temp_subject = result["s"]["value"]
                id = result["id"]["value"]
                node_subject = temp_subject.replace(id, "")
                if(node_subject not in self.top_node_list_uri):
                    self.top_node_list_uri[node_subject] = []
                    self.listbox_gpm_uri_list.insert(index, node_subject)
                    # container ep listに追加
                    self.con_ep_list.append(node_subject)
                    index += 1
                self.top_node_list_uri[node_subject].append(node_id)

    def set_gpm(self):
        """ treeviewとグラフで確認したいGPMを設定

        """
        selected_GPM_index = self.listbox_gpm_uri_list.curselection()
        if(len(selected_GPM_index) == 0):
            messagebox.showerror("Error", "No data selected!", parent=self)
        else:
            self.gpm_uri = self.listbox_gpm_uri_list.get(selected_GPM_index)

            # treeviewを更新のため、元のtreeviewを削除
            if(self.treeview.get_children):
                for item in self.treeview.get_children():
                    self.treeview.delete(item)

            # treeviewを表示
            self.get_hierarchy_node_list()

            # グラフを表示
            self.show_top_win2_graph()

    def win2_tree_select(self, event):
        """ treeviewを選択処理

        Args:
            event: ツリーリスト選択イベント

        """
        # node名を取得
        record_id = self.treeview.focus()
        current_item = self.treeview.item(record_id)["text"]

        # node idを取得
        for key in self.treeview_item_list.keys():
            for index, item in enumerate(self.treeview_item_list[key]):
                if(item == record_id):
                    node_id = self.node_id_list[key][index]
                    new_graph_level = key

        image_path = "selected_win2_graph.png"
        # 選択したノードの層と表示しているグラフの層が異なる場合、グラフ更新
        if(new_graph_level != self.graph_level):
            self.graph_level = new_graph_level

            (self.graph_node_list, self.graph_edge_list, \
                self.graph_label_list, self.graph_position) = \
                self.data.get_graph_data_by_ttlfile(self.gpm_uri, \
                                        self.node_id_list[self.graph_level], \
                                        self.object_id_list[self.graph_level], \
                                        None, self.ttl_file_win2)

        # 画像を設定
        set_image(image_path, self.graph_node_list, self.graph_edge_list, \
                self.graph_label_list, [node_id], \
                self.graph_position)

        # 画像を更新
        selected_image = Image.open(image_path)
        self.change_image(selected_image)

    def get_hierarchy_node_list(self):
        """ ツリーリスト階層ノード情報設定

        """
        # グラフにあるコンテナフロー数を取得
        container_count = self.data.get_graph_containerflow_count(
            self.selected_graph_name, self.gpm_uri)

        (self.node_id_list) = self.data.get_top_node_list_in_uri(
            self.selected_graph_name, self.gpm_uri)

        (self.object_list, self.object_id_list) = \
                                self.data.get_top_level_object_list(
                                    self.selected_graph_name, self.gpm_uri)

        self.node_list = {}
        self.node_list["top"] = []
        self.treeview_item_list = {}
        self.treeview_item_list["top"] = []

        for node_id in self.node_id_list["top"]:
            node_value = self.data.get_node_value(self.selected_graph_name, \
                self.gpm_uri, node_id)
            self.node_list["top"].append(node_value)

        for index, node in enumerate(self.node_list["top"]):
            parent = self.treeview.insert("", tk.END, text=node, open=False)
            self.treeview_item_list["top"].append(parent)
            self.get_member_node_list(
                self.gpm_uri, self.node_id_list["top"][index], container_count, parent)

    def get_member_node_list(self, uri, top_node, count, parent_node):
        """ ツリーリストmember情報を取得

        Args:
            uri: 選択したGPMのuri
            top_node: member情報を取得したいトップノードのid
            count: GPMにあるコンテナ数
            parent_node: ツリーリストの親ノード

        Returns:

        """
        if(count == 0):
            return

        top_node = top_node.replace("<br>", "")

        # Containerを取得
        query_result = self.data.get_container_query(self.selected_graph_name, \
                                                uri + top_node)

        if(len(query_result) > 0):
            self.node_list[top_node] = []
            self.node_id_list[top_node] = []
            self.object_list[top_node] = []
            self.object_id_list[top_node] = []
            self.treeview_item_list[top_node] = []
            count -= 1
            for result in query_result:
                container = result["source"]["value"]

            # Containerのメンバーを取得
            query_result = self.data.get_container_member_query(
                            self.selected_graph_name, container, uri)

            for result in query_result:
                value = result["value"]["value"]
                id = result["id"]["value"]
                self.node_list[top_node].append(value)
                self.node_id_list[top_node].append(id)
                # return self.get_member_node_list(value, count)

            # Containerにあるオブジェクトを取得
            query_object_result = self.data.get_obejct_in_container_query(
                                    self.selected_graph_name, container, uri)

            for result in query_object_result:
                id = result["id"]["value"]
                value = result["value"]["value"]
                self.object_list[top_node].append(value)
                self.object_id_list[top_node].append(id)

            for index, node in enumerate(self.node_list[top_node]):
                child = self.treeview.insert(
                    parent_node, tk.END, text=node, open=False)
                self.treeview_item_list[top_node].append(child)
                node_uri = self.data.get_node_uri(
                    self.selected_graph_name, self.node_id_list[top_node][index])
                self.get_member_node_list(
                    node_uri, self.node_id_list[top_node][index], count, child)
        else:
            return

    def show_top_win2_graph(self):
        """ グラフのTOP部分を表示

        """
        # グラフ情報を取得
        (self.graph_node_list, self.graph_edge_list, \
            self.graph_label_list, self.graph_position) = \
            self.data.get_graph_data_by_ttlfile(self.gpm_uri, 
                                    self.node_id_list["top"], \
                                    self.object_id_list["top"], None, \
                                    self.ttl_file_win2)

        # 最上層グラフを表示
        image_path = "top_win2_graph.png"
        self.graph_level = "top"
        set_image(image_path, self.graph_node_list, self.graph_edge_list, \
                self.graph_label_list, [], \
                self.graph_position)
        top_image = Image.open(image_path)

        self.graph_canvas.photo = ImageTk.PhotoImage(top_image)
        self.image_on_canvas = self.graph_canvas.create_image(
            -100, -100, anchor='nw', image=self.graph_canvas.photo)

    def change_image(self, img):
        """ グラフ画像の入り替え処理

        Args:
            img: 更新する画像

        """
        # canvasの書き換え
        self.graph_canvas.photo = ImageTk.PhotoImage(img)
        self.graph_canvas.itemconfig(
            self.image_on_canvas, image=self.graph_canvas.photo)

    def win2_ep_select(self):
        """ EP選択処理

        """
        selected_GPM_index = self.listbox_gpm_uri_list.curselection()

        if(len(selected_GPM_index) == 0):
            messagebox.showerror("Error", "プロセスを選択していません!", parent=self)
        else:
            self.gpm_uri = self.listbox_gpm_uri_list.get(selected_GPM_index)
            # 主ウインドウに設定する
            self.win2_graph.set(self.selected_graph_name)
            self.win2_uri.set(self.gpm_uri)

    def win2_node_select(self):
        """ ノード選択処理

        """
        # node名を取得
        record_id = self.treeview.focus()
        if(len(record_id) == 0):
            messagebox.showerror("Error", "コンテナを選択していません!", parent=self)
        else:
            current_item = self.treeview.item(record_id)["text"]

            # node idを取得
            for key in self.treeview_item_list.keys():
                for index, item in enumerate(self.treeview_item_list[key]):
                    if(item == record_id):
                        node_id = self.node_id_list[key][index]

            # 選択したノードのuriを取得
            node_uri = self.data.get_node_uri(self.selected_graph_name, node_id)

            # 主ウインドウに設定する
            self.win2_graph.set(self.selected_graph_name)
            self.win2_uri.set(node_uri)
            self.win2_selected_item.set(node_id)

    def renew_widget(self):
        """ widget更新処理

        """
        # treeviewを更新のため、元のtreeviewを削除
        if(self.treeview.get_children):
            for item in self.treeview.get_children():
                self.treeview.delete(item)

        # gpm listの選択をクリア
        self.listbox_gpm_uri_list.delete(0, tk.END)

        (self.node_id_list) = \
                self.data.get_top_node_list_in_graph(self.selected_graph_name)

        self.get_uri_list(self.node_id_list)

        # canvasクリア
        self.graph_canvas.delete("all")

In [ ]:
def set_image(image_path, nodes, edges, labels, select_node, node_postion):
    """ グラフ画像を作成

    Args:
        image_path: 画像保存パス
        nodes: ノード情報
        edges: エッジ情報
        labels: ラベル情報
        select_node: 選択したノード
        node_postion:  位置情報

    """
    graph = nx.DiGraph()
    graph.clear()
    plt.clf()

    graph_nodes = copy.deepcopy(nodes)
    for item in graph_nodes:
        item_list = list(item)
        if(item_list[0] in select_node):
            item_list[1]["color"] = "orange"

    # 頂点を設定
    graph.add_nodes_from(graph_nodes)
    # 辺を設定
    graph.add_edges_from(edges)

    edge_color = [edge["color"] for edge in graph.edges.values()]
    nodeShapes = set((aShape[1]["s"] for aShape in graph.nodes(data=True)))

    # graph size
    plt.figure(3, figsize=(10, 10))
    #plt.margins(0.5, 0)

    # node_size default=300
    for aShape in nodeShapes:
        nx.draw_networkx_nodes(graph, pos=node_postion, node_shape=aShape, \
                            nodelist=[sNode[0] for sNode in filter(
                                lambda x: x[1]["s"] == aShape, graph.nodes(data=True))], \
                            node_color=[sNode[1]["color"] for sNode in filter(
                                lambda x: x[1]["s"] == aShape, graph.nodes(data=True))], \
                            node_size=500, alpha=0.5)

    nx.draw_networkx_edges(graph, pos=node_postion, \
                        edge_color=edge_color, width=2.0, alpha=0.5)

    nx.draw_networkx_labels(graph, node_postion, labels, font_size=9, \
                            font_family="IPAexGothic", \
                            font_color="black")
    # 枠線disable
    plt.gca().spines["right"].set_visible(False)
    plt.gca().spines["top"].set_visible(False)
    plt.gca().spines["left"].set_visible(False)
    plt.gca().spines["bottom"].set_visible(False)

    # 保存
    plt.savefig(image_path)


In [ ]:
if __name__ == "__main__":
    """ メイン関数

    """
    # Windowを生成する
    root = tk.Tk()

    s=ttk.Style()
    if platform.system() == "Windows":
        s.configure("style.TFrame" ,bd="2", relief="groove")
    else: 
        s.theme_use('default')

    app = Application(master=root)
    # Windowをループさせて、継続的にWindow表示させる
    app.mainloop()
